In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math


In [4]:
import random
import numpy as np
from numpy import ndarray

In [67]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 9 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [6]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/CO2_QM_ds_mod_augmented.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [7]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (MB, SQ), (SB, LQ), (SB, SQ), (MB, LQ), (SB, SQ), (SB, MQ),
     (SB, MQ), (SB, MQ), (MB, SQ), (SB, SQ), (SB, MQ), (LB, SQ), (MB, MQ), (SB, LQ), (MB, MQ)
 ]

In [79]:
# Compute total number of required points across all test combinations
required_points = sum(data_size for data_size, _ in test_combinations)

# Sample from the full dataset without replacement
if required_points > total_points:
    raise ValueError(f"Need {required_points} points but only {total_points} available.")

sampled_indices = np.random.choice(total_points, size=required_points, replace=False)
combined_dataset = dataset[sampled_indices]


In [78]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


In [ ]:
print(len(queries_list))
print(len(combined_dataset))

15
1590000


**-------------Cover Tree---------------**

**Izbicki's implementation**

In [ ]:
import numpy as np
from typing import Optional, List, Tuple
from dataclasses import dataclass

@dataclass
class Node:
    point: np.ndarray
    level: int
    children: List['Node']
    _maxdist_cache: Optional[float] = None  # Cache for maxdist

    @property
    def covdist(self) -> float:
        """Coverage distance of the node."""
        return 2.0 ** self.level

    @property
    def maxdist(self) -> float:
        """Maximum distance within the subtree rooted at this node."""
        if self._maxdist_cache is None:
            if not self.children:
                self._maxdist_cache = 0
            else:
                self._maxdist_cache = max(
                    self.covdist,
                    max(child.maxdist + np.linalg.norm(self.point - child.point)
                        for child in self.children)
                )
        return self._maxdist_cache

    def update_maxdist(self):
        """Reset and recompute maxdist cache."""
        self._maxdist_cache = None
        _ = self.maxdist  # Force recomputation

class CoverTree:
    def __init__(self):
        self.root: Optional[Node] = None

    def insert(self, x: np.ndarray) -> None:
        """Insert a point into the cover tree."""
        if self.root is None:
            self.root = Node(x, 0, [])
            return

        d = np.linalg.norm(self.root.point - x)
        if d > self.root.covdist:
            # Precompute levels to expand
            levels_to_expand = int(np.ceil(np.log2(d / self.root.covdist)))
            for _ in range(levels_to_expand):
                self.root = Node(self.root.point, self.root.level + 1, [self.root])
            self.root = Node(x, self.root.level + 1, [self.root])
            return

        self._insert_recursive(self.root, x)

    def _insert_recursive(self, p: Node, x: np.ndarray) -> bool:
        """Recursive insertion into the cover tree."""
        for child in p.children:
            if np.linalg.norm(child.point - x) <= child.covdist:
                if self._insert_recursive(child, x):
                    p.update_maxdist()
                    return True

        # If x fits within the current node's coverage
        if np.linalg.norm(p.point - x) <= p.covdist:
            p.children.append(Node(x, p.level - 1, []))
            p.update_maxdist()
            return True

        return False

    def find_nearest(self, x: np.ndarray) -> Tuple[np.ndarray, float]:
        """Find the nearest neighbor to the query point."""
        if self.root is None:
            raise ValueError("Tree is empty")

        return self._find_nearest_recursive(self.root, x, self.root)

    def _find_nearest_recursive(self, p: Node, x: np.ndarray, best: Node) -> Tuple[np.ndarray, float]:
        """Recursive nearest neighbor search with pruning."""
        d_p = np.linalg.norm(p.point - x)
        d_best = np.linalg.norm(best.point - x)

        # Update the best point if the current node is closer
        if d_p < d_best:
            best = p
            d_best = d_p

        # Sort children by distance to the query point
        for child in sorted(p.children, key=lambda c: np.linalg.norm(c.point - x)):
            d_x_to_child = np.linalg.norm(child.point - x)

            # Prune subtrees based on the pruning condition
            if d_best > d_x_to_child - child.maxdist:
                child_point, child_dist = self._find_nearest_recursive(child, x, best)
                if child_dist < d_best:
                    best = Node(child_point, p.level, [])
                    d_best = child_dist

        return best.point, d_best


------------

**----------CV Run 1----------------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 223.08it/s]


Build Time: 24.88 sec, Search Time: 0.09 sec

1: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:02<00:00, 242.28it/s]


Build Time: 2.93 sec, Search Time: 2.05 sec

2: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 42.08 sec, Search Time: 1.34 sec

13: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:07<00:00, 71.20it/s]


Build Time: 4.15 sec, Search Time: 6.98 sec

14: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 66.09it/s]

Build Time: 42.30 sec, Search Time: 1.50 sec

=== Summary of CV Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 24.88 sec, Search: 0.09 sec
Data:   10000 points, Queries:  500 => Build: 2.93 sec, Search: 2.05 sec
Data:   10000 points, Queries:   20 => Build: 3.04 sec, Search: 0.09 sec
Data:  100000 points, Queries:  500 => Build: 31.50 sec, Search: 2.92 sec
Data:   10000 points, Queries:   20 => Build: 3.60 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 3.24 sec, Search: 0.65 sec
Data:   10000 points, Queries:  100 => Build: 3.31 sec, Search: 0.62 sec
Data:   10000 points, Queries:  100 => Build: 3.38 sec, Search: 0.64 sec
Data:  100000 points, Queries:   20 => Build: 34.41 sec, Search: 0.19 sec
Data:   10000 points, Queries:   20 => Build: 3.48 sec, Search: 0.16 sec
Data:   10000 points, Queries:  100 => Build: 3.50 sec, Search: 0.72 sec
Data: 1000000 points, Queries:   20 => Build: 396.00 sec, Search: 0.26 sec
Data:  100000 points, Queries:  100 => B

**----CV Run 2-------------**

In [ ]:
# Test code CV Tree

results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
cv_tree = CoverTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the CV tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        cv_tree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        nn_pt_cv, r_min_cv = cv_tree.find_nearest(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")
# ----------------------------
# Summary of Total times
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of CV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


0: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 269.35it/s]


Build Time: 24.74 sec, Search Time: 0.07 sec

1: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:01<00:00, 259.05it/s]


Build Time: 2.85 sec, Search Time: 1.92 sec

2: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 40.39 sec, Search Time: 1.23 sec

13: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:05<00:00, 86.09it/s]


Build Time: 4.03 sec, Search Time: 5.77 sec

14: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 81.64it/s]

Build Time: 41.55 sec, Search Time: 1.22 sec

=== Summary of CV Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 24.74 sec, Search: 0.07 sec
Data:   10000 points, Queries:  500 => Build: 2.85 sec, Search: 1.92 sec
Data:   10000 points, Queries:   20 => Build: 2.84 sec, Search: 0.07 sec
Data:  100000 points, Queries:  500 => Build: 29.94 sec, Search: 2.60 sec
Data:   10000 points, Queries:   20 => Build: 3.13 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 3.15 sec, Search: 0.53 sec
Data:   10000 points, Queries:  100 => Build: 3.13 sec, Search: 0.58 sec
Data:   10000 points, Queries:  100 => Build: 3.20 sec, Search: 0.53 sec
Data:  100000 points, Queries:   20 => Build: 32.41 sec, Search: 0.14 sec
Data:   10000 points, Queries:   20 => Build: 3.57 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 3.34 sec, Search: 0.68 sec
Data: 1000000 points, Queries:   20 => Build: 379.07 sec, Search: 0.24 sec
Data:  100000 points, Queries:  100 => B

**----------CV Analysis-------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 620.11',
    'Total Time: 593.14'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 606.6250 sec, Std Dev: 19.0707 sec, CV: 3.14%


-----------------

**--------------------------VP_MWV-----------------------------**

In [ ]:
import numpy as np

class VPNode:
    def __init__(self, vp=None):
        # Vantage point
        self.vp = vp

        # Distance bounds
        self.low_near = float('inf')
        self.low_far = 0
        self.high_near = float('inf')
        self.high_far = 0

        # Child nodes
        self.lowChild = None
        self.highChild = None

        # Bucket for leaf nodes
        self.bucket = []

    def is_leaf(self):
        return self.lowChild is None and self.highChild is None

class VPTree:
    def __init__(self, bucket_size=50, distance_fn=None):
        """
        Initialize a VP-Tree

        Parameters:
        - bucket_size: Square root of maximum bucket size (b where b² is max bucket size)
        - distance_fn: Distance function to use (defaults to Euclidean)
        """
        self.bucket_size = bucket_size
        self.root = None

        # Default to Euclidean distance if none provided
        self.distance_fn = distance_fn if distance_fn is not None else self._euclidean_distance

    def _euclidean_distance(self, x, y):
        """Calculate Euclidean distance between two points"""
        x = np.asarray(x)
        y = np.asarray(y)
        diff = x - y
        return np.sqrt(np.dot(diff, diff))

    def insert(self, x):
        """Insert a new datapoint x into the VP-tree"""
        # If tree is empty, create root with this point as VP
        if self.root is None:
            self.root = VPNode(vp=x)
            return

        p = self.root

        # Traverse until we reach a leaf node
        while not p.is_leaf():
            # Calculate distance from x to current vantage point
            dist = self.distance_fn(x, p.vp)

            # Determine which child to go to
            if dist < (p.low_far + p.high_near) / 2:
                # Update distance bounds
                p.low_far = max(dist, p.low_far)
                p.low_near = min(dist, p.low_near)

                # Go to low (near) child
                p = p.lowChild
            else:
                # Update distance bounds
                p.high_far = max(dist, p.high_far)
                p.high_near = min(dist, p.high_near)

                # Go to high (far) child
                p = p.highChild

        # Add x to bucket leaf node p
        p.bucket.append(x)

        # If bucket exceeds b², split it
        if len(p.bucket) > self.bucket_size**2:
            self._split_node(p)

    def _select_vantage_point(self, points):
        """Select a vantage point from the bucket
        Can use various methods; simple random selection is shown here.
        """
        # For simplicity, using the first point as VP
        # Could implement more sophisticated selection methods
        if points:
            return points[0]
        return None

    def _split_node(self, p):
        """Split a node using the CART-inspired weighted variance method"""
        points = p.bucket.copy()

        # Select vantage point from bucket
        vp = self._select_vantage_point(points)
        points.remove(vp)  # Remove VP from points list

        # Calculate distances to vantage point
        distances = [(self.distance_fn(vp, point), point) for point in points]

        # Sort by distance
        distances.sort(key=lambda x: x[0])
        sorted_points = [point for _, point in distances]
        sorted_dists = [dist for dist, _ in distances]

        # Find optimal split using weighted variance minimization
        n = len(sorted_points)
        best_split = 0
        min_weighted_variance = float('inf')

        for s in range(1, n):
            # Calculate variances for 1:s and s:n
            if s > 0:
                var_left = np.var(sorted_dists[:s]) if s > 1 else 0
            else:
                var_left = 0

            if n-s > 0:
                var_right = np.var(sorted_dists[s:]) if n-s > 1 else 0
            else:
                var_right = 0

            # Calculate weighted variance
            weighted_var = s * var_left + (n-s) * var_right

            if weighted_var < min_weighted_variance:
                min_weighted_variance = weighted_var
                best_split = s

        # Create new children nodes
        p.lowChild = VPNode(vp=vp)
        p.highChild = VPNode(vp=self._select_vantage_point(sorted_points[best_split:]))

        # Distribute points to children based on best split
        p.lowChild.bucket = sorted_points[:best_split]
        p.highChild.bucket = sorted_points[best_split:]

        # Initialize distance bounds
        if p.lowChild.bucket:
            low_dists = sorted_dists[:best_split]
            p.low_near = min(low_dists)
            p.low_far = max(low_dists)

        if p.highChild.bucket:
            # If we have a vantage point for the high child, calculate distances to it
            high_dists = [self.distance_fn(p.highChild.vp, point) for point in p.highChild.bucket]
            p.high_near = min(high_dists) if high_dists else float('inf')
            p.high_far = max(high_dists) if high_dists else 0

        # Clear the bucket as it's no longer a leaf
        p.bucket = []

    def nearest_neighbor(self, query):
        """Optimized nearest neighbor search with batch processing for bucket points"""
        best_dist = float('inf')
        best_point = None

        # Use a stack instead of recursion to reduce overhead
        # Each entry is (node, min_possible_dist)
        stack = [(self.root, 0)]

        while stack:
            node, min_dist = stack.pop()

            # Skip this node if it cannot contain a better solution
            if min_dist >= best_dist:
                continue

            # Check vantage point distance
            if node.vp is not None:
                # Calculate distance to vantage point
                diff = np.subtract(query, node.vp)
                dist_to_vp = np.sqrt(np.dot(diff, diff))

                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.vp

            # If leaf node, check all points in bucket using batch processing
            if node.is_leaf():
                if node.bucket:
                    # Convert bucket to numpy array once
                    bucket_array = np.array(node.bucket)

                    # Vectorized distance calculation
                    query_array = np.array(query)
                    diffs = bucket_array - query_array
                    squared_dists = np.sum(diffs**2, axis=1)

                    # Find minimum distance point without unnecessary sqrt operations
                    min_idx = np.argmin(squared_dists)
                    min_squared_dist = squared_dists[min_idx]

                    # Only compute sqrt for potentially better distance
                    if min_squared_dist < best_dist**2:
                        min_dist = np.sqrt(min_squared_dist)
                        if min_dist < best_dist:
                            best_dist = min_dist
                            best_point = node.bucket[min_idx]
                continue

            # Calculate threshold
            threshold = (node.low_far + node.high_near) / 2

            # Calculate bounds for both children
            if dist_to_vp < threshold:
                # Query is in near region
                # Add far child to stack first (so near child gets processed first)
                if node.highChild:
                    far_bound = max(0, dist_to_vp - node.low_far)
                    if far_bound < best_dist:
                        stack.append((node.highChild, far_bound))

                # Add near child with zero bound (must contain query)
                if node.lowChild:
                    stack.append((node.lowChild, 0))
            else:
                # Query is in far region
                # Add near child to stack first (so far child gets processed first)
                if node.lowChild:
                    near_bound = max(0, node.high_near - dist_to_vp)
                    if near_bound < best_dist:
                        stack.append((node.lowChild, near_bound))

                # Add far child with zero bound (must contain query)
                if node.highChild:
                    stack.append((node.highChild, 0))

        return best_dist, best_point

------------

**VP_MWV (Run 1)**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 195.71it/s]


Build Time: 20.19 sec, Search Time: 0.10 sec

2: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1291.91it/s]


Build Time: 3.10 sec, Search Time: 0.38 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 22.53 sec, Search Time: 0.09 sec

14: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1032.13it/s]


Build Time: 0.71 sec, Search Time: 0.48 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 951.50it/s]

Build Time: 20.82 sec, Search Time: 0.10 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 20.19 sec, Search: 0.10 sec
Data:   10000 points, Queries:  500 => Build: 3.10 sec, Search: 0.38 sec
Data:   10000 points, Queries:   20 => Build: 2.17 sec, Search: 0.02 sec
Data:  100000 points, Queries:  500 => Build: 21.92 sec, Search: 0.55 sec
Data:   10000 points, Queries:   20 => Build: 3.49 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 2.17 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 1.54 sec, Search: 0.08 sec
Data:   10000 points, Queries:  100 => Build: 2.51 sec, Search: 0.08 sec
Data:  100000 points, Queries:   20 => Build: 21.83 sec, Search: 0.02 sec
Data:   10000 points, Queries:   20 => Build: 2.63 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 4.19 sec, Search: 0.09 sec
Data: 1000000 points, Queries:   20 => Build: 235.82 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => Buil

**-----VP_MWV Run 2-------------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 193.91it/s]


Build Time: 23.47 sec, Search Time: 0.10 sec

2: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1013.34it/s]


Build Time: 2.81 sec, Search Time: 0.49 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 21.87 sec, Search Time: 0.11 sec

14: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 983.75it/s]


Build Time: 2.02 sec, Search Time: 0.50 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 963.51it/s]

Build Time: 24.61 sec, Search Time: 0.10 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 23.47 sec, Search: 0.10 sec
Data:   10000 points, Queries:  500 => Build: 2.81 sec, Search: 0.49 sec
Data:   10000 points, Queries:   20 => Build: 1.19 sec, Search: 0.02 sec
Data:  100000 points, Queries:  500 => Build: 24.08 sec, Search: 0.54 sec
Data:   10000 points, Queries:   20 => Build: 2.22 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 1.58 sec, Search: 0.08 sec
Data:   10000 points, Queries:  100 => Build: 0.90 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 2.89 sec, Search: 0.08 sec
Data:  100000 points, Queries:   20 => Build: 19.57 sec, Search: 0.03 sec
Data:   10000 points, Queries:   20 => Build: 3.87 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 0.93 sec, Search: 0.10 sec
Data: 1000000 points, Queries:   20 => Build: 237.71 sec, Search: 0.04 sec
Data:  100000 points, Queries:  100 => Buil

**-------VP_MWV Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases

vptree = VPTree()

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}: === Testing with Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[st_idx:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = vptree.nearest_neighbor(query=query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of VP MWV Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1: === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 508.82it/s]


Build Time: 21.11 sec, Search Time: 0.04 sec

2: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1061.92it/s]


Build Time: 1.86 sec, Search Time: 0.47 sec

3: === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 26.21 sec, Search Time: 0.09 sec

14: === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:00<00:00, 1026.25it/s]


Build Time: 3.43 sec, Search Time: 0.48 sec

15: === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:00<00:00, 848.63it/s]

Build Time: 23.77 sec, Search Time: 0.12 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 21.11 sec, Search: 0.04 sec
Data:   10000 points, Queries:  500 => Build: 1.86 sec, Search: 0.47 sec
Data:   10000 points, Queries:   20 => Build: 2.86 sec, Search: 0.02 sec
Data:  100000 points, Queries:  500 => Build: 22.42 sec, Search: 0.61 sec
Data:   10000 points, Queries:   20 => Build: 2.95 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 4.39 sec, Search: 0.11 sec
Data:   10000 points, Queries:  100 => Build: 1.63 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 2.65 sec, Search: 0.10 sec
Data:  100000 points, Queries:   20 => Build: 22.11 sec, Search: 0.03 sec
Data:   10000 points, Queries:   20 => Build: 2.59 sec, Search: 0.02 sec
Data:   10000 points, Queries:  100 => Build: 1.29 sec, Search: 0.09 sec
Data: 1000000 points, Queries:   20 => Build: 243.26 sec, Search: 0.03 sec
Data:  100000 points, Queries:  100 => Buil

**-------VP_MWV Analysis------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 367.79',
    'Total Time: 372.06',
    'Total Time: 384.86'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 374.9033 sec, Std Dev: 8.8831 sec, CV: 2.37%


----------

**------------------Ball Tree--------------------**

In [ ]:
import numpy as np
from collections import deque

#*-----------------------------------------------------------------------------
class BallNode:
    def __init__(self):
        self.center = None
        self.radius = None
        self.left = None
        self.right = None
        self.points = None
        self.pivot1 = None
        self.pivot2 = None

#*-----------------------------------------------------------------------------
class BallTree:
    def __init__(self, leaf_size=40):
        self.leaf_size = leaf_size
        self.root = None

    #*-----------------------------------------------------------------------------
    def _find_furthest_points(self, points):
        # First find point furthest from origin as first pivot
        distances = np.sum(points ** 2, axis=1)
        p1_idx = np.argmax(distances)
        pivot1 = points[p1_idx]

        # Find point furthest from first pivot
        distances = np.sum((points - pivot1) ** 2, axis=1)
        p2_idx = np.argmax(distances)
        pivot2 = points[p2_idx]

        return pivot1, pivot2

    #*-----------------------------------------------------------------------------
    def _compute_distances(self, points, pivot):
        return np.sqrt(np.sum((points - pivot) ** 2, axis=1))

    #*-----------------------------------------------------------------------------
    def build(self, points):
        if points is None or len(points) == 0:
            return None

        points = np.array(points)
        self.root = self._build_recursive(points)
        return self.root

    #*-----------------------------------------------------------------------------
    def _build_recursive(self, points):
        if len(points) <= self.leaf_size:
            node = BallNode()
            node.points = points
            node.center = np.mean(points, axis=0)
            if len(points) > 1:
                node.radius = np.max(self._compute_distances(points, node.center))
            else:
                node.radius = 0
            return node

        node = BallNode()

        # Find two furthest points as pivots
        pivot1, pivot2 = self._find_furthest_points(points)
        node.pivot1 = pivot1
        node.pivot2 = pivot2

        # Assign points to closest pivot
        dist_to_p1 = self._compute_distances(points, pivot1)
        dist_to_p2 = self._compute_distances(points, pivot2)

        left_mask = dist_to_p1 <= dist_to_p2
        left_points = points[left_mask]
        right_points = points[~left_mask]

        # Compute node properties
        node.center = np.mean(points, axis=0)
        node.radius = np.max(self._compute_distances(points, node.center))

        # Recursively build subtrees
        node.left = self._build_recursive(left_points)
        node.right = self._build_recursive(right_points)

        return node

    #*-----------------------------------------------------------------------------
    def search(self, point):
        if self.root is None:
            return float('inf'), None

        point = np.array(point)
        best_dist = float('inf')
        best_point = None

        # Use a priority queue for best-first search
        # Each entry is (lower_bound_distance, node)
        queue = deque([(0, self.root)])

        while queue:
            _, node = queue.popleft()  # Get the node with lowest lower bound distance

            # Calculate distance from query point to node center
            dist_to_center = np.linalg.norm(point - node.center)

            # If the closest possible point in this ball is farther than our best,
            # we can skip this subtree
            if dist_to_center - node.radius > best_dist:
                continue

            if node.points is not None:
                # Leaf node - check all points
                distances = np.linalg.norm(node.points - point, axis=1)
                min_idx = np.argmin(distances)
                min_dist = distances[min_idx]

                if min_dist < best_dist:
                    best_dist = min_dist
                    best_point = node.points[min_idx]
            else:
                # Internal node - decide which child to visit first
                dist_to_p1 = np.linalg.norm(point - node.pivot1)
                dist_to_p2 = np.linalg.norm(point - node.pivot2)

                # Calculate lower bound distances to both subtrees
                # and visit them in order of potential closeness
                if dist_to_p1 <= dist_to_p2:
                    # Visit left subtree first, then right
                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        queue.appendleft((left_lb, node.left))  # Add to front to visit next

                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        if right_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((right_lb, node.right))
                else:
                    # Visit right subtree first, then left
                    if node.right:
                        right_lb = max(0, dist_to_center - node.right.radius)
                        queue.appendleft((right_lb, node.right))  # Add to front to visit next

                    if node.left:
                        left_lb = max(0, dist_to_center - node.left.radius)
                        if left_lb <= best_dist:  # Only add if it could contain a closer point
                            queue.append((left_lb, node.left))

        return best_dist, best_point
    #*-----------------------------------------------------------------------------

    def insert(self, point):
        """
        Insert a new point into the Ball Tree.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        # Path to keep track of nodes to update
        path = []

        # Traverse the tree to find the appropriate leaf
        current = self.root
        path.append(current)

        while current.points is None:  # Continue until we reach a leaf node
            # Calculate distances to pivots
            dist_to_p1 = np.sqrt(np.sum((point - current.pivot1) ** 2))
            dist_to_p2 = np.sqrt(np.sum((point - current.pivot2) ** 2))

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                    path.append(current)
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update centers and radii of nodes on the path
                    self._update_ancestors(path, point)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid)
        # New centroid = (n * old_centroid + new_point) / (n + 1)
        current.center = (n * current.center + point) / (n + 1)

        # Update the radius if necessary
        dist_to_center = np.sqrt(np.sum((point - current.center) ** 2))
        current.radius = max(current.radius, dist_to_center)

        # Update centers and radii of ancestor nodes
        path.pop()  # Remove the leaf node as we've already updated it
        self._update_ancestors(path, point)

    #*-----------------------------------------------------------------------------
    def _update_ancestors(self, path, point):
        """
        Update the center and radius of all nodes in the path after insertion.

        Parameters:
            path (list): List of nodes from root to leaf (excluding the leaf)
            point (np.ndarray): The point that was inserted
        """
        # Process nodes from bottom to top (leaf's parent first)
        for node in reversed(path):
            # For center, we would ideally recompute based on all points
            # but as an approximation, we'll slightly shift it toward the new point
            # This way we avoid having to recursively gather all points

            # We'll apply a small weight to the new point based on estimated subtree size
            # Assume a balanced tree with approximately 2^depth nodes at the bottom
            depth = len(path) - path.index(node)
            weight = 1.0 / (2**depth)

            # Update center with minimal shift toward new point
            node.center = (1 - weight) * node.center + weight * point

            # Check if radius needs update by calculating distance from new point to center
            dist_to_center = np.sqrt(np.sum((point - node.center) ** 2))

            # Only update radius if the new point is outside the current ball
            if dist_to_center > node.radius:
                node.radius = dist_to_center

    #*-----------------------------------------------------------------------------
    def insert_opt(self, point):
        """
        Insert a new point into the Ball Tree with optimized performance.

        Parameters:
            point (np.ndarray): The point to insert.
        """
        point = np.array(point)

        # Handle empty tree
        if self.root is None:
            self.root = BallNode()
            self.root.points = np.array([point])
            self.root.center = point.copy()
            self.root.radius = 0
            return

        current = self.root
        nodes_to_update = []  # Store nodes that need radius updates

        # Traverse the tree to find the appropriate leaf
        while current.points is None:  # Continue until we reach a leaf node
            # Check if point is within current node's radius
            dist_to_center = np.linalg.norm(point - current.center)
            if dist_to_center > current.radius:
                nodes_to_update.append((current, dist_to_center))

            # Calculate distances to pivots (only once per level)
            dist_to_p1 = np.linalg.norm(point - current.pivot1)
            dist_to_p2 = np.linalg.norm(point - current.pivot2)

            if dist_to_p1 <= dist_to_p2:
                # Go to left child (closer to pivot1)
                if current.left:
                    current = current.left
                else:
                    # Create new leaf node if doesn't exist
                    current.left = BallNode()
                    current.left.points = np.array([point])
                    current.left.center = point.copy()
                    current.left.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return
            else:
                # Go to right child (closer to pivot2)
                if current.right:
                    current = current.right
                else:
                    # Create new leaf node if doesn't exist
                    current.right = BallNode()
                    current.right.points = np.array([point])
                    current.right.center = point.copy()
                    current.right.radius = 0

                    # Update only radii of nodes that need updating
                    for node, dist in nodes_to_update:
                        node.radius = max(node.radius, dist)
                    return

        # We've reached a leaf node
        n = len(current.points)

        # Add the point to the leaf
        current.points = np.vstack([current.points, point])

        # Update the center (centroid) - avoid creating temporary arrays
        current.center *= n
        current.center += point
        current.center /= (n + 1)

        # Update the radius if necessary
        dist_to_center = np.linalg.norm(point - current.center)
        if dist_to_center > current.radius:
            current.radius = dist_to_center

        # Update only radii of nodes that need updates
        for node, dist in nodes_to_update:
            node.radius = max(node.radius, dist)
#*-----------------------------------------------------------------------------

--------

**--------Ball Tree (Run 1)----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 381.78it/s]


Build Time: 0.20 sec, Search Time: 0.05 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:01<00:00, 352.27it/s]


Build Time: 0.19 sec, Search Time: 1.41 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.57 sec, Search Time: 1.18 sec

14:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:05<00:00, 83.92it/s]


Build Time: 4.50 sec, Search Time: 5.91 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 78.18it/s]

Build Time: 4.77 sec, Search Time: 1.27 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.20 sec, Search: 0.05 sec
Data:   10000 points, Queries:  500 => Build: 0.19 sec, Search: 1.41 sec
Data:   10000 points, Queries:   20 => Build: 0.20 sec, Search: 0.06 sec
Data:  100000 points, Queries:  500 => Build: 0.44 sec, Search: 2.41 sec
Data:   10000 points, Queries:   20 => Build: 0.47 sec, Search: 0.11 sec
Data:   10000 points, Queries:  100 => Build: 0.49 sec, Search: 0.55 sec
Data:   10000 points, Queries:  100 => Build: 0.51 sec, Search: 0.48 sec
Data:   10000 points, Queries:  100 => Build: 0.53 sec, Search: 0.53 sec
Data:  100000 points, Queries:   20 => Build: 0.87 sec, Search: 0.13 sec
Data:   10000 points, Queries:   20 => Build: 0.90 sec, Search: 0.13 sec
Data:   10000 points, Queries:  100 => Build: 0.94 sec, Search: 0.65 sec
Data: 1000000 points, Queries:   20 => Build: 4.39 sec, Search: 0.24 sec
Data:  100000 points, Queries:  100 => Build: 4.5

**-------Ball Run 2----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 315.66it/s]


Build Time: 0.16 sec, Search Time: 0.06 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:01<00:00, 304.80it/s]


Build Time: 0.18 sec, Search Time: 1.63 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.48 sec, Search Time: 1.25 sec

14:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 81.07it/s]


Build Time: 4.51 sec, Search Time: 6.12 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 78.69it/s]

Build Time: 4.89 sec, Search Time: 1.26 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.16 sec, Search: 0.06 sec
Data:   10000 points, Queries:  500 => Build: 0.18 sec, Search: 1.63 sec
Data:   10000 points, Queries:   20 => Build: 0.19 sec, Search: 0.08 sec
Data:  100000 points, Queries:  500 => Build: 0.45 sec, Search: 2.43 sec
Data:   10000 points, Queries:   20 => Build: 0.48 sec, Search: 0.09 sec
Data:   10000 points, Queries:  100 => Build: 0.50 sec, Search: 0.53 sec
Data:   10000 points, Queries:  100 => Build: 0.51 sec, Search: 0.53 sec
Data:   10000 points, Queries:  100 => Build: 0.56 sec, Search: 0.51 sec
Data:  100000 points, Queries:   20 => Build: 0.82 sec, Search: 0.12 sec
Data:   10000 points, Queries:   20 => Build: 0.83 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 0.86 sec, Search: 0.58 sec
Data: 1000000 points, Queries:   20 => Build: 4.11 sec, Search: 0.22 sec
Data:  100000 points, Queries:  100 => Build: 4.4

**---------Ball Tree Run 3----------**

In [ ]:
results = []
st_idx = 0  # reset the dataset slicing index for processing test cases
#* Create the Tree
leaf_size = 4050

for idx, (data_size, query_size) in enumerate(test_combinations):
    print(f"\n{idx+1}:  === Testing with Data Size: {data_size:,} points, Query Size: {query_size} queries ===")

    # Slice the current data chunk for this test.
    end_idx = st_idx + data_size
    current_data = combined_dataset[:end_idx]
    st_idx = end_idx

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[idx]

    # --- Build the Ball tree ---
    build_start = time.perf_counter()
    btree = BallTree(leaf_size=leaf_size)
    btree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    search_time = 0.0
    for query in tqdm(queries, desc="Querying"):
        search_start = time.perf_counter()
        r_min, nn_pt = btree.search(query)
        search_time += time.perf_counter() - search_start

    results.append(((data_size, query_size), build_time, search_time))
    print(f"Build Time: {build_time:.2f} sec, Search Time: {search_time:.2f} sec")

# ----------------------------
# Summary of Results
# ----------------------------
print("\n=== Summary of Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    print(f"Data: {data_size:>7} points, Queries: {query_size:>4} => Build: {b_time:.2f} sec, Search: {s_time:.2f} sec")

print("--------------------------------------------")
# ----------------------------
# Total Results
# ----------------------------
build_time_total = search_time_total = 0.0
print("\n=== Summary of Ball Tree Tests ===")
for (data_size, query_size), b_time, s_time in results:
    build_time_total += b_time
    search_time_total += s_time
print(f"\nTotal Build Time: {build_time_total:.2f} sec, Total Search Time: {search_time_total:.2f} sec")

print("--------------------------------------------")
total_time = build_time_total + search_time_total
print(f"Total Time: {total_time:.2f}")


1:  === Testing with Data Size: 100,000 points, Query Size: 20 queries ===


Querying: 100%|██████████| 20/20 [00:00<00:00, 325.65it/s]


Build Time: 0.16 sec, Search Time: 0.06 sec

2:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:01<00:00, 293.63it/s]


Build Time: 0.18 sec, Search Time: 1.69 sec

3:  === Testing with Data Size: 10,000 points, Query Size: 20 queries ===


...

Build Time: 4.32 sec, Search Time: 1.30 sec

14:  === Testing with Data Size: 10,000 points, Query Size: 500 queries ===


Querying: 100%|██████████| 500/500 [00:06<00:00, 82.24it/s]


Build Time: 4.32 sec, Search Time: 6.04 sec

15:  === Testing with Data Size: 100,000 points, Query Size: 100 queries ===


Querying: 100%|██████████| 100/100 [00:01<00:00, 75.04it/s]

Build Time: 4.71 sec, Search Time: 1.32 sec

=== Summary of Tree Tests ===
Data:  100000 points, Queries:   20 => Build: 0.16 sec, Search: 0.06 sec
Data:   10000 points, Queries:  500 => Build: 0.18 sec, Search: 1.69 sec
Data:   10000 points, Queries:   20 => Build: 0.19 sec, Search: 0.08 sec
Data:  100000 points, Queries:  500 => Build: 0.43 sec, Search: 2.53 sec
Data:   10000 points, Queries:   20 => Build: 0.46 sec, Search: 0.10 sec
Data:   10000 points, Queries:  100 => Build: 0.48 sec, Search: 0.52 sec
Data:   10000 points, Queries:  100 => Build: 0.51 sec, Search: 0.51 sec
Data:   10000 points, Queries:  100 => Build: 0.53 sec, Search: 0.54 sec
Data:  100000 points, Queries:   20 => Build: 0.84 sec, Search: 0.14 sec
Data:   10000 points, Queries:   20 => Build: 0.90 sec, Search: 0.12 sec
Data:   10000 points, Queries:  100 => Build: 0.90 sec, Search: 0.59 sec
Data: 1000000 points, Queries:   20 => Build: 3.96 sec, Search: 0.21 sec
Data:  100000 points, Queries:  100 => Build: 4.3

**--------Ball Tree Analysis--------------**

In [ ]:
import re
import numpy as np

# The 3 result strings
results = [
    'Total Time: 39.08',
    'Total Time: 39.06',
    'Total Time: 38.65'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time:\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 38.9300 sec, Std Dev: 0.2427 sec, CV: 0.62%


----------------

**----------QuickSelect Algorithm-----------**

In [10]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

-------------------------------

**--------KD Hybrid---------------------**

In [11]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

--------------------------

**-----------KD Hybrid Sqrt Run 1----------**

In [14]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.6056 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 327.36it/s]


Total Search Time: 0.088107 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 130196.02it/s]


Insert Time: 0.0787 sec


...

Total Search Time: 22.373622 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 110408.96it/s]


Insert Time: 0.9087 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.34it/s]

Total Search Time: 4.942201 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.6056 sec, Total Search Time: 0.088107 sec, Total Time: 0.6937 sec
Points: 110,000, Insert Time: 0.0787 sec, Total Search Time: 1.581722 sec, Total Time: 1.6604 sec
Points: 120,000, Insert Time: 0.0828 sec, Total Search Time: 0.104729 sec, Total Time: 0.1876 sec
Points: 220,000, Build Time: 1.1345 sec, Total Search Time: 3.262027 sec, Total Time: 4.3965 sec
Points: 230,000, Insert Time: 0.1056 sec, Total Search Time: 0.198628 sec, Total Time: 0.3042 sec
Points: 240,000, Insert Time: 0.1203 sec, Total Search Time: 0.731537 sec, Total Time: 0.8518 sec
Points: 250,000, Insert Time: 0.0967 sec, Total Search Time: 0.751567 sec, Total Time: 0.8483 sec
Points: 260,000, Insert Time: 0.0990 sec, Total Search Time: 0.784776 sec, Total Time: 0.8837 sec
Points: 360,000, Insert Time: 0.7865 sec, Total Search Time: 0.303179 sec, Total Time: 1.0897 sec
Points: 370,000, Insert Time: 0.1102 sec, Total

**----------KD Hybrid Sqrt Run 2-----------**

In [17]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5848 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 347.40it/s]


Total Search Time: 0.089721 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 119114.40it/s]

Insert Time: 0.0879 sec


...

Total Search Time: 22.507795 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 116040.29it/s]


Insert Time: 0.8648 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 21.78it/s]

Total Search Time: 5.048881 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5848 sec, Total Search Time: 0.089721 sec, Total Time: 0.6745 sec
Points: 110,000, Insert Time: 0.0879 sec, Total Search Time: 1.585060 sec, Total Time: 1.6730 sec
Points: 120,000, Insert Time: 0.0844 sec, Total Search Time: 0.099946 sec, Total Time: 0.1843 sec
Points: 220,000, Build Time: 1.1444 sec, Total Search Time: 3.222973 sec, Total Time: 4.3674 sec
Points: 230,000, Insert Time: 0.0866 sec, Total Search Time: 0.196406 sec, Total Time: 0.2831 sec
Points: 240,000, Insert Time: 0.0794 sec, Total Search Time: 0.720109 sec, Total Time: 0.7996 sec
Points: 250,000, Insert Time: 0.0820 sec, Total Search Time: 0.773826 sec, Total Time: 0.8558 sec
Points: 260,000, Insert Time: 0.0888 sec, Total Search Time: 0.743880 sec, Total Time: 0.8327 sec
Points: 360,000, Insert Time: 0.8011 sec, Total Search Time: 0.312226 sec, Total Time: 1.1133 sec
Points: 370,000, Insert Time: 0.0810 sec, Total

**----------KD Hybrid Sqrt Run 3-----------**

In [22]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5458 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 335.39it/s]


Total Search Time: 0.089903 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 125055.28it/s]

Insert Time: 0.0840 sec


...

Total Search Time: 22.641940 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 113697.93it/s]


Insert Time: 0.8833 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.54it/s]

Total Search Time: 4.896449 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5458 sec, Total Search Time: 0.089903 sec, Total Time: 0.6357 sec
Points: 110,000, Insert Time: 0.0840 sec, Total Search Time: 1.513518 sec, Total Time: 1.5976 sec
Points: 120,000, Insert Time: 0.0869 sec, Total Search Time: 0.098942 sec, Total Time: 0.1858 sec
Points: 220,000, Build Time: 1.3795 sec, Total Search Time: 3.263256 sec, Total Time: 4.6428 sec
Points: 230,000, Insert Time: 0.1024 sec, Total Search Time: 0.203690 sec, Total Time: 0.3061 sec
Points: 240,000, Insert Time: 0.0817 sec, Total Search Time: 0.752738 sec, Total Time: 0.8345 sec
Points: 250,000, Insert Time: 0.0908 sec, Total Search Time: 0.743229 sec, Total Time: 0.8340 sec
Points: 260,000, Insert Time: 0.1089 sec, Total Search Time: 0.749212 sec, Total Time: 0.8582 sec
Points: 360,000, Insert Time: 0.8390 sec, Total Search Time: 0.300932 sec, Total Time: 1.1400 sec
Points: 370,000, Insert Time: 0.0812 sec, Total

**------------KD Hybrid Sqrt analysis----------**

In [27]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 56.3593',
    'Total Time:: 55.8922',
    'Total Time:: 56.1730'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 56.1415 sec, Std Dev: 0.2351 sec, CV: 0.42%


-----------------------------------

**------KD Hybrid Log Run 1--------**

In [12]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4754 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 289.32it/s]


Total Search Time: 0.099992 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 102960.55it/s]

Insert Time: 0.1005 sec


...

Total Search Time: 22.543552 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 117992.93it/s]


Insert Time: 0.8507 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.01it/s]

Total Search Time: 4.983666 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4754 sec, Total Search Time: 0.099992 sec, Total Time: 0.5754 sec
Points: 110,000, Insert Time: 0.1005 sec, Total Search Time: 1.578445 sec, Total Time: 1.6789 sec
Points: 120,000, Insert Time: 0.1161 sec, Total Search Time: 0.103252 sec, Total Time: 0.2194 sec
Points: 220,000, Insert Time: 0.8267 sec, Total Search Time: 2.369134 sec, Total Time: 3.1958 sec
Points: 230,000, Insert Time: 0.1015 sec, Total Search Time: 0.210976 sec, Total Time: 0.3124 sec
Points: 240,000, Insert Time: 0.0833 sec, Total Search Time: 0.585709 sec, Total Time: 0.6690 sec
Points: 250,000, Insert Time: 0.1056 sec, Total Search Time: 0.610177 sec, Total Time: 0.7158 sec
Points: 260,000, Insert Time: 0.0883 sec, Total Search Time: 0.656923 sec, Total Time: 0.7452 sec
Points: 360,000, Insert Time: 0.8352 sec, Total Search Time: 0.318722 sec, Total Time: 1.1540 sec
Points: 370,000, Insert Time: 0.0854 sec, Tota

**-------KD Hybrid Log Run 2-----------**

In [18]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5890 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 370.84it/s]


Total Search Time: 0.084997 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 126680.16it/s]


Insert Time: 0.0821 sec


...

Total Search Time: 22.551091 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 112767.11it/s]


Insert Time: 0.8902 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 21.78it/s]

Total Search Time: 5.039899 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5890 sec, Total Search Time: 0.084997 sec, Total Time: 0.6740 sec
Points: 110,000, Insert Time: 0.0821 sec, Total Search Time: 1.566077 sec, Total Time: 1.6482 sec
Points: 120,000, Insert Time: 0.0921 sec, Total Search Time: 0.122740 sec, Total Time: 0.2149 sec
Points: 220,000, Insert Time: 0.8582 sec, Total Search Time: 2.361182 sec, Total Time: 3.2193 sec
Points: 230,000, Insert Time: 0.0910 sec, Total Search Time: 0.192218 sec, Total Time: 0.2832 sec
Points: 240,000, Insert Time: 0.0865 sec, Total Search Time: 0.587619 sec, Total Time: 0.6741 sec
Points: 250,000, Insert Time: 0.0975 sec, Total Search Time: 0.635976 sec, Total Time: 0.7335 sec
Points: 260,000, Insert Time: 0.0862 sec, Total Search Time: 0.638923 sec, Total Time: 0.7251 sec
Points: 360,000, Insert Time: 0.8326 sec, Total Search Time: 0.317681 sec, Total Time: 1.1503 sec
Points: 370,000, Insert Time: 0.0798 sec, Tota

**------KD Hybrid Log Run 3------**

In [23]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.6600 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 401.45it/s]


Total Search Time: 0.080171 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 123954.31it/s]


Insert Time: 0.0842 sec


...

Total Search Time: 22.618027 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 115710.91it/s]


Insert Time: 0.8673 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.35it/s]

Total Search Time: 4.935784 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.6600 sec, Total Search Time: 0.080171 sec, Total Time: 0.7402 sec
Points: 110,000, Insert Time: 0.0842 sec, Total Search Time: 1.549698 sec, Total Time: 1.6339 sec
Points: 120,000, Insert Time: 0.0827 sec, Total Search Time: 0.101123 sec, Total Time: 0.1838 sec
Points: 220,000, Insert Time: 0.8610 sec, Total Search Time: 2.364015 sec, Total Time: 3.2250 sec
Points: 230,000, Insert Time: 0.1124 sec, Total Search Time: 0.209966 sec, Total Time: 0.3224 sec
Points: 240,000, Insert Time: 0.1068 sec, Total Search Time: 0.616408 sec, Total Time: 0.7232 sec
Points: 250,000, Insert Time: 0.0777 sec, Total Search Time: 0.626015 sec, Total Time: 0.7038 sec
Points: 260,000, Insert Time: 0.0917 sec, Total Search Time: 0.631638 sec, Total Time: 0.7234 sec
Points: 360,000, Insert Time: 0.8145 sec, Total Search Time: 0.325208 sec, Total Time: 1.1397 sec
Points: 370,000, Insert Time: 0.1073 sec, Tota

**---------KD Hybrid Log Analysis----------**

In [26]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 54.6540',
    'Total Time:: 54.1586',
    'Total Time:: 54.5070'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 54.4399 sec, Std Dev: 0.2544 sec, CV: 0.47%


-----------------------------

**--------KD Hybrid Log Ratio Run 1---------**

In [13]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4650 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 351.61it/s]


Total Search Time: 0.085122 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 120014.88it/s]


Insert Time: 0.0867 sec


...

Total Search Time: 22.487983 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 113389.63it/s]


Insert Time: 0.8841 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 21.68it/s]

Total Search Time: 5.082725 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4650 sec, Total Search Time: 0.085122 sec, Total Time: 0.5501 sec
Points: 110,000, Insert Time: 0.0867 sec, Total Search Time: 1.537585 sec, Total Time: 1.6243 sec
Points: 120,000, Insert Time: 0.1175 sec, Total Search Time: 0.104436 sec, Total Time: 0.2219 sec
Points: 220,000, Insert Time: 0.9384 sec, Total Search Time: 2.378843 sec, Total Time: 3.3172 sec
Points: 230,000, Insert Time: 0.0962 sec, Total Search Time: 0.211038 sec, Total Time: 0.3072 sec
Points: 240,000, Insert Time: 0.0912 sec, Total Search Time: 0.590758 sec, Total Time: 0.6820 sec
Points: 250,000, Insert Time: 0.0818 sec, Total Search Time: 0.573323 sec, Total Time: 0.6551 sec
Points: 260,000, Insert Time: 0.0819 sec, Total Search Time: 0.615884 sec, Total Time: 0.6978 sec
Points: 360,000, Insert Time: 0.8137 sec, Total Search Time: 0.314704 sec, Total Time: 1.1284 sec
Points: 370,000, Insert Time: 0.0751 sec, Tota

**------KD Hybrid Log Ratio Run 2---------**

In [19]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.3886 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 371.85it/s]


Total Search Time: 0.083724 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 88603.88it/s]

Insert Time: 0.1156 sec


...

Total Search Time: 22.567392 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 115683.49it/s]


Insert Time: 0.8682 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.05it/s]

Total Search Time: 4.956388 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.3886 sec, Total Search Time: 0.083724 sec, Total Time: 0.4723 sec
Points: 110,000, Insert Time: 0.1156 sec, Total Search Time: 1.542847 sec, Total Time: 1.6584 sec
Points: 120,000, Insert Time: 0.0798 sec, Total Search Time: 0.106134 sec, Total Time: 0.1860 sec
Points: 220,000, Insert Time: 0.7999 sec, Total Search Time: 2.304513 sec, Total Time: 3.1044 sec
Points: 230,000, Insert Time: 0.0885 sec, Total Search Time: 0.197335 sec, Total Time: 0.2859 sec
Points: 240,000, Insert Time: 0.0910 sec, Total Search Time: 0.559027 sec, Total Time: 0.6501 sec
Points: 250,000, Insert Time: 0.0943 sec, Total Search Time: 0.599821 sec, Total Time: 0.6942 sec
Points: 260,000, Insert Time: 0.1007 sec, Total Search Time: 0.612546 sec, Total Time: 0.7133 sec
Points: 360,000, Insert Time: 0.8713 sec, Total Search Time: 0.313846 sec, Total Time: 1.1851 sec
Points: 370,000, Insert Time: 0.0868 sec, Tota

**------KD Hybrid Log Ratio Run 3-----**

In [24]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 2050
epsilon_rebuild = 0.235

leaf_size_insert = 2100
epsilon_insert = 0.07

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.leaf_size = leaf_size_insert
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5801 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 369.41it/s]


Total Search Time: 0.084647 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 128391.43it/s]


Insert Time: 0.0801 sec


...

Total Search Time: 22.411368 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:00<00:00, 113554.97it/s]


Insert Time: 0.8858 sec


Querying batch 15: 100%|██████████| 100/100 [00:04<00:00, 22.71it/s]

Total Search Time: 4.845984 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5801 sec, Total Search Time: 0.084647 sec, Total Time: 0.6647 sec
Points: 110,000, Insert Time: 0.0801 sec, Total Search Time: 1.562647 sec, Total Time: 1.6427 sec
Points: 120,000, Insert Time: 0.0952 sec, Total Search Time: 0.116311 sec, Total Time: 0.2115 sec
Points: 220,000, Insert Time: 0.7905 sec, Total Search Time: 2.423182 sec, Total Time: 3.2137 sec
Points: 230,000, Insert Time: 0.0815 sec, Total Search Time: 0.208897 sec, Total Time: 0.2904 sec
Points: 240,000, Insert Time: 0.0843 sec, Total Search Time: 0.608600 sec, Total Time: 0.6929 sec
Points: 250,000, Insert Time: 0.0813 sec, Total Search Time: 0.668360 sec, Total Time: 0.7496 sec
Points: 260,000, Insert Time: 0.0807 sec, Total Search Time: 0.646170 sec, Total Time: 0.7268 sec
Points: 360,000, Insert Time: 0.8431 sec, Total Search Time: 0.320817 sec, Total Time: 1.1639 sec
Points: 370,000, Insert Time: 0.0781 sec, Tota

**----------KD Hybrid Log Ratio Analysis--------**

In [25]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 54.7460',
    'Total Time:: 54.1006',
    'Total Time:: 53.9938'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 54.2801 sec, Std Dev: 0.4070 sec, CV: 0.75%


---------------------------

**----------KD Tree Epsilon Dynamic---------**

In [28]:
import numpy as np
from collections import deque
from typing import List, Tuple, Union

#*-----------------------------------------------------------------------------
class KDNode:
    def __init__(self, points=None, left=None, right=None, parent=None):
        self.points = points if points is not None else []  # Now always a list of points
        self.split_dim = None
        self.split_val = None
        self.leaf = False
        self.left_tr = left
        self.right_tr = right
        self.parent = parent
        self.centroid = None
        self.radius = 0
        # NumPy array cache for search operations
        self._np_points = None

#*-----------------------------------------------------------------------------
class KDTree:
    def __init__(self, leaf_size=40, epsilon=0.1):
        self.points = None
        self.root = None
        self.dim = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def find_best_split_dimension(self, points: np.ndarray, start_dim: int) -> Tuple[int, bool]:
        if len(points) == 0:
            return 0, True
        for d in range(self.dim):
            curr_dim = (start_dim + d) % self.dim
            if not np.all(points[:, curr_dim] == points[0, curr_dim]):
                return curr_dim, False
        return 0, True

    #*-----------------------------------------------------------------------------
    def build(self, points=None):
        self.points = points
        self.dim = points.shape[1]
        self.root = KDNode()
        stack = deque([(self.root, points, 0)])

        while stack:
            node, node_points, depth = stack.pop()

            if len(node_points) <= self.leaf_size:
                node.leaf = True
                node.points = node_points.tolist() # Convert to list of points
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            split_dim, all_equal = self.find_best_split_dimension(node_points, depth % self.dim)
            if all_equal:
                node.leaf = True
                node.points = node_points.tolist()
                node.centroid = np.mean(node_points, axis=0)
                if len(node_points) > 1:
                    node.radius = np.max(np.linalg.norm(node_points - node.centroid, axis=1))
                continue

            node.split_dim = split_dim
            coords = [(p[split_dim], i) for i, p in enumerate(node_points)]
            k = len(node_points) // 2
            med_ = quickselect(coords, k)
            median_val = med_[0]
            node.split_val = median_val

            left_points = node_points[node_points[:, split_dim] < (median_val - self.epsilon)]
            right_points = node_points[node_points[:, split_dim] > (median_val + self.epsilon)]
            middle_points = node_points[
                (node_points[:, split_dim] >= (median_val - self.epsilon)) &
                (node_points[:, split_dim] <= (median_val + self.epsilon))
            ]

            node.points = [p for p in middle_points]
            if len(middle_points) > 0:
                node.centroid = np.mean(middle_points, axis=0)
                if len(middle_points) > 1:
                    node.radius = np.max(np.linalg.norm(middle_points - node.centroid, axis=1))

            if len(left_points) > 0:
                node.left_tr = KDNode(parent=node)
                stack.append((node.left_tr, left_points, depth + 1))
            if len(right_points) > 0:
                node.right_tr = KDNode(parent=node)
                stack.append((node.right_tr, right_points, depth + 1))
            if len(left_points) == 0 and len(right_points) == 0:
                node.leaf = True

        return self.root

    #*-----------------------------------------------------------------------------
    def insert(self, point: Union[List, np.ndarray]):
        if self.root is None:
            self.dim = len(point)
            self.points = [point]
            self.root = KDNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            self.root.leaf = True
            return

        current = self.root

        while not current.leaf:
            split_dim = current.split_dim
            split_val = current.split_val

            if (len(current.points) > 0 and len(current.points) < self.leaf_size
                and abs(point[split_dim] - split_val) <= self.epsilon):
                n = len(current.points)
                current.points.append(point)
                current.centroid = (n * current.centroid + point) / (n + 1)
                dist_to_centroid = np.linalg.norm(point - current.centroid)
                current.radius = max(current.radius, dist_to_centroid)
                # Clear the NumPy cache since we modified the points
                current._np_points = None
                return

            if point[split_dim] < split_val - self.epsilon:
                if current.left_tr is None:
                    current.left_tr = KDNode(parent=current)
                    current.left_tr.points = [point]
                    current.left_tr.centroid = point
                    current.left_tr.radius = 0
                    current.left_tr.leaf = True
                    return
                current = current.left_tr
            else:
                if current.right_tr is None:
                    current.right_tr = KDNode(parent=current)
                    current.right_tr.points = [point]
                    current.right_tr.centroid = point
                    current.right_tr.radius = 0
                    current.right_tr.leaf = True
                    return
                current = current.right_tr

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous data storage
        if n == 2*self.leaf_size:
            self.leaf_size = n
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        dist_to_centroid = np.linalg.norm(point - current.centroid)
        current.radius = max(current.radius, dist_to_centroid)
        # Clear the NumPy cache since we modified the points
        current._np_points = None

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based point collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Search for the nearest neighbor of the query point using KD-tree.
        Uses triangle inequality pruning at leaf nodes.

        Parameters:
            query (np.ndarray): The query point

        Returns:
            Tuple of (distance to nearest neighbor, nearest neighbor point)
        """

        best_dist = float('inf')
        best_point = None

        # Stack for depth-first traversal
        stack = [self.root]

        while stack:
            node = stack.pop()

            if node.leaf:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius >= best_dist:
                    continue

                # Check if _np_points exists, if not, create it
                if node._np_points is None and node.points:
                    node._np_points = np.array(node.points)

                # Use the cached NumPy array for distance calculations
                if node._np_points is not None:
                    dists = np.linalg.norm(node._np_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_points[min_idx]
                continue

            if node.split_dim is not None:
                if len(node.points) > 0:
                    # Check if _np_points exists, if not, create it
                    if node._np_points is None and node.points:
                        node._np_points = np.array(node.points)

                    # Use the cached NumPy array for distance calculations
                    if node._np_points is not None:
                        dists = np.linalg.norm(node._np_points - query, axis=1)
                        min_idx = np.argmin(dists)
                        if dists[min_idx] < best_dist:
                            best_dist = dists[min_idx]
                            best_point = node._np_points[min_idx]

                split_dim = node.split_dim
                split_val = node.split_val
                diff = query[split_dim] - split_val

                if diff < -self.epsilon:
                    if node.right_tr and abs(diff) < best_dist:
                        stack.append(node.right_tr)
                    if node.left_tr:
                        stack.append(node.left_tr)
                elif diff > self.epsilon:
                    if node.left_tr and abs(diff) < best_dist:
                        stack.append(node.left_tr)
                    if node.right_tr:
                        stack.append(node.right_tr)
                else:
                    if abs(diff) <= self.epsilon:
                        if diff < 0:
                            if node.left_tr:
                                stack.append(node.left_tr)
                            if node.right_tr:
                                stack.append(node.right_tr)
                        else:
                            if node.right_tr:
                                stack.append(node.right_tr)
                            if node.left_tr:
                                stack.append(node.left_tr)

        return best_dist, best_point
#*-----------------------------------------------------------------------------

------------------------

**----------kd dyn sqrt run 1-----------**

In [29]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.7649 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 480.93it/s]


Total Search Time: 0.087911 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 106967.53it/s]

Insert Time: 0.0964 sec


...

Total Search Time: 16.473480 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 88329.37it/s]


Insert Time: 1.1352 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 37.68it/s]

Total Search Time: 3.247126 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.7649 sec, Total Search Time: 0.087911 sec, Total Time: 0.8528 sec
Points: 110,000, Insert Time: 0.0964 sec, Total Search Time: 1.210537 sec, Total Time: 1.3069 sec
Points: 120,000, Insert Time: 0.0919 sec, Total Search Time: 0.100686 sec, Total Time: 0.1926 sec
Points: 220,000, Build Time: 1.5298 sec, Total Search Time: 2.276278 sec, Total Time: 3.8060 sec
Points: 230,000, Insert Time: 0.1122 sec, Total Search Time: 0.173731 sec, Total Time: 0.2860 sec
Points: 240,000, Insert Time: 0.1107 sec, Total Search Time: 0.583312 sec, Total Time: 0.6940 sec
Points: 250,000, Insert Time: 0.1113 sec, Total Search Time: 0.564367 sec, Total Time: 0.6757 sec
Points: 260,000, Insert Time: 0.1238 sec, Total Search Time: 0.588323 sec, Total Time: 0.7121 sec
Points: 360,000, Insert Time: 0.9175 sec, Total Search Time: 0.287585 sec, Total Time: 1.2051 sec
Points: 370,000, Insert Time: 0.0937 sec, Total

**---------Kd dyn sqrt run 2---------**

In [35]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.8292 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 453.79it/s]


Total Search Time: 0.089151 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 100085.76it/s]

Insert Time: 0.1031 sec


...

Total Search Time: 16.677070 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 85278.87it/s]


Insert Time: 1.1755 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 36.51it/s]

Total Search Time: 3.293391 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.8292 sec, Total Search Time: 0.089151 sec, Total Time: 0.9183 sec
Points: 110,000, Insert Time: 0.1031 sec, Total Search Time: 1.204198 sec, Total Time: 1.3073 sec
Points: 120,000, Insert Time: 0.1105 sec, Total Search Time: 0.110809 sec, Total Time: 0.2213 sec
Points: 220,000, Build Time: 1.6074 sec, Total Search Time: 2.311040 sec, Total Time: 3.9184 sec
Points: 230,000, Insert Time: 0.0979 sec, Total Search Time: 0.194975 sec, Total Time: 0.2928 sec
Points: 240,000, Insert Time: 0.1074 sec, Total Search Time: 0.577981 sec, Total Time: 0.6854 sec
Points: 250,000, Insert Time: 0.1175 sec, Total Search Time: 0.650515 sec, Total Time: 0.7680 sec
Points: 260,000, Insert Time: 0.1193 sec, Total Search Time: 0.649864 sec, Total Time: 0.7691 sec
Points: 360,000, Insert Time: 1.0348 sec, Total Search Time: 0.323304 sec, Total Time: 1.3581 sec
Points: 370,000, Insert Time: 0.1080 sec, Total

**--------Kd dyn sqrt run 3-----------**

In [40]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.7105 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 447.16it/s]


Total Search Time: 0.084739 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 104928.69it/s]

Insert Time: 0.0993 sec


...

Total Search Time: 18.980872 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 85149.08it/s]


Insert Time: 1.1776 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.00it/s]

Total Search Time: 3.860858 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.7105 sec, Total Search Time: 0.084739 sec, Total Time: 0.7952 sec
Points: 110,000, Insert Time: 0.0993 sec, Total Search Time: 1.301700 sec, Total Time: 1.4010 sec
Points: 120,000, Insert Time: 0.0866 sec, Total Search Time: 0.103773 sec, Total Time: 0.1904 sec
Points: 220,000, Build Time: 2.1334 sec, Total Search Time: 2.638456 sec, Total Time: 4.7719 sec
Points: 230,000, Insert Time: 0.1345 sec, Total Search Time: 0.220432 sec, Total Time: 0.3549 sec
Points: 240,000, Insert Time: 0.1115 sec, Total Search Time: 0.649997 sec, Total Time: 0.7615 sec
Points: 250,000, Insert Time: 0.1041 sec, Total Search Time: 0.640691 sec, Total Time: 0.7448 sec
Points: 260,000, Insert Time: 0.1296 sec, Total Search Time: 0.648034 sec, Total Time: 0.7777 sec
Points: 360,000, Insert Time: 1.0124 sec, Total Search Time: 0.308330 sec, Total Time: 1.3207 sec
Points: 370,000, Insert Time: 0.0944 sec, Total

**-------Kd dyn sqrt analysis------------**

In [45]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 49.4123',
    'Total Time:: 50.7168',
    'Total Time:: 55.5564'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 51.8952 sec, Std Dev: 3.2371 sec, CV: 6.24%


----------------------------

**-------KD  Dyn Log Threshold Run 1-----**

In [30]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.4548 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 525.68it/s]


Total Search Time: 0.074863 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 106918.18it/s]

Insert Time: 0.0967 sec


...

Total Search Time: 16.340479 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 92665.92it/s]


Insert Time: 1.0831 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 37.67it/s]

Total Search Time: 3.211194 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.4548 sec, Total Search Time: 0.074863 sec, Total Time: 0.5296 sec
Points: 110,000, Insert Time: 0.0967 sec, Total Search Time: 1.172548 sec, Total Time: 1.2693 sec
Points: 120,000, Insert Time: 0.0849 sec, Total Search Time: 0.096238 sec, Total Time: 0.1811 sec
Points: 220,000, Insert Time: 0.8734 sec, Total Search Time: 1.744811 sec, Total Time: 2.6183 sec
Points: 230,000, Insert Time: 0.0896 sec, Total Search Time: 0.191468 sec, Total Time: 0.2810 sec
Points: 240,000, Insert Time: 0.0895 sec, Total Search Time: 0.480683 sec, Total Time: 0.5702 sec
Points: 250,000, Insert Time: 0.0879 sec, Total Search Time: 0.498213 sec, Total Time: 0.5861 sec
Points: 260,000, Insert Time: 0.0868 sec, Total Search Time: 0.517659 sec, Total Time: 0.6044 sec
Points: 360,000, Insert Time: 0.9715 sec, Total Search Time: 0.301402 sec, Total Time: 1.2729 sec
Points: 370,000, Insert Time: 0.0895 sec, Tota

**-----KD Dyn Log Run 2--------**

In [36]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.0366 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 464.12it/s]


Total Search Time: 0.087107 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 127299.17it/s]

Insert Time: 0.0819 sec


...

Total Search Time: 16.771002 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 89818.81it/s]


Insert Time: 1.1167 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 36.12it/s]

Total Search Time: 3.428609 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.0366 sec, Total Search Time: 0.087107 sec, Total Time: 1.1237 sec
Points: 110,000, Insert Time: 0.0819 sec, Total Search Time: 1.251452 sec, Total Time: 1.3333 sec
Points: 120,000, Insert Time: 0.0940 sec, Total Search Time: 0.116468 sec, Total Time: 0.2105 sec
Points: 220,000, Insert Time: 0.9391 sec, Total Search Time: 1.880812 sec, Total Time: 2.8199 sec
Points: 230,000, Insert Time: 0.1293 sec, Total Search Time: 0.216593 sec, Total Time: 0.3459 sec
Points: 240,000, Insert Time: 0.0992 sec, Total Search Time: 0.539640 sec, Total Time: 0.6389 sec
Points: 250,000, Insert Time: 0.0930 sec, Total Search Time: 0.525019 sec, Total Time: 0.6180 sec
Points: 260,000, Insert Time: 0.0897 sec, Total Search Time: 0.607557 sec, Total Time: 0.6972 sec
Points: 360,000, Insert Time: 0.9739 sec, Total Search Time: 0.366255 sec, Total Time: 1.3402 sec
Points: 370,000, Insert Time: 0.1076 sec, Tota

**-----KD Dyn Log Run 3------**

In [41]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 1.0842 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 411.69it/s]


Total Search Time: 0.098660 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 102094.91it/s]

Insert Time: 0.1013 sec


...

Total Search Time: 18.819289 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 78633.05it/s]


Insert Time: 1.2755 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 30.96it/s]

Total Search Time: 3.815303 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 1.0842 sec, Total Search Time: 0.098660 sec, Total Time: 1.1828 sec
Points: 110,000, Insert Time: 0.1013 sec, Total Search Time: 1.390649 sec, Total Time: 1.4920 sec
Points: 120,000, Insert Time: 0.1012 sec, Total Search Time: 0.136783 sec, Total Time: 0.2380 sec
Points: 220,000, Insert Time: 0.9571 sec, Total Search Time: 2.032535 sec, Total Time: 2.9896 sec
Points: 230,000, Insert Time: 0.1068 sec, Total Search Time: 0.228527 sec, Total Time: 0.3353 sec
Points: 240,000, Insert Time: 0.0892 sec, Total Search Time: 0.540934 sec, Total Time: 0.6302 sec
Points: 250,000, Insert Time: 0.0881 sec, Total Search Time: 0.607230 sec, Total Time: 0.6954 sec
Points: 260,000, Insert Time: 0.0938 sec, Total Search Time: 0.654737 sec, Total Time: 0.7485 sec
Points: 360,000, Insert Time: 0.9709 sec, Total Search Time: 0.355540 sec, Total Time: 1.3265 sec
Points: 370,000, Insert Time: 0.0864 sec, Tota

**---------KD Dyn Log Analysis---------**

In [44]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 47.9060',
    'Total Time:: 50.3845',
    'Total Time:: 54.5511'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 50.9472 sec, Std Dev: 3.3581 sec, CV: 6.59%


-----------------------

**------KD Tree Dynamic Log Ratio Run 1--------**

In [31]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.9154 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 482.05it/s]


Total Search Time: 0.087188 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 103233.48it/s]

Insert Time: 0.0994 sec


...

Total Search Time: 16.136747 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 84904.01it/s]


Insert Time: 1.1811 sec


Querying batch 15: 100%|██████████| 100/100 [00:02<00:00, 38.04it/s]

Total Search Time: 3.196067 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.9154 sec, Total Search Time: 0.087188 sec, Total Time: 1.0026 sec
Points: 110,000, Insert Time: 0.0994 sec, Total Search Time: 1.210849 sec, Total Time: 1.3102 sec
Points: 120,000, Insert Time: 0.0932 sec, Total Search Time: 0.108618 sec, Total Time: 0.2018 sec
Points: 220,000, Insert Time: 0.8817 sec, Total Search Time: 1.883773 sec, Total Time: 2.7655 sec
Points: 230,000, Insert Time: 0.0958 sec, Total Search Time: 0.214221 sec, Total Time: 0.3100 sec
Points: 240,000, Insert Time: 0.0896 sec, Total Search Time: 0.530866 sec, Total Time: 0.6205 sec
Points: 250,000, Insert Time: 0.0979 sec, Total Search Time: 0.590457 sec, Total Time: 0.6884 sec
Points: 260,000, Insert Time: 0.0834 sec, Total Search Time: 0.552553 sec, Total Time: 0.6359 sec
Points: 360,000, Insert Time: 0.8754 sec, Total Search Time: 0.311084 sec, Total Time: 1.1865 sec
Points: 370,000, Insert Time: 0.0975 sec, Tota

**--------KD Dyn Log Ratio Run 2------**

In [37]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.9466 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 411.28it/s]


Total Search Time: 0.094589 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 79125.87it/s]


Insert Time: 0.1296 sec


...

Total Search Time: 16.606152 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 85291.27it/s]


Insert Time: 1.1758 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.98it/s]

Total Search Time: 3.697480 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.9466 sec, Total Search Time: 0.094589 sec, Total Time: 1.0412 sec
Points: 110,000, Insert Time: 0.1296 sec, Total Search Time: 1.350632 sec, Total Time: 1.4803 sec
Points: 120,000, Insert Time: 0.0961 sec, Total Search Time: 0.113928 sec, Total Time: 0.2100 sec
Points: 220,000, Insert Time: 0.9899 sec, Total Search Time: 2.263394 sec, Total Time: 3.2533 sec
Points: 230,000, Insert Time: 0.1250 sec, Total Search Time: 0.238573 sec, Total Time: 0.3636 sec
Points: 240,000, Insert Time: 0.0947 sec, Total Search Time: 0.603949 sec, Total Time: 0.6986 sec
Points: 250,000, Insert Time: 0.1093 sec, Total Search Time: 0.613886 sec, Total Time: 0.7232 sec
Points: 260,000, Insert Time: 0.0887 sec, Total Search Time: 0.626249 sec, Total Time: 0.7149 sec
Points: 360,000, Insert Time: 0.9348 sec, Total Search Time: 0.388365 sec, Total Time: 1.3232 sec
Points: 370,000, Insert Time: 0.0922 sec, Tota

**-------KD Dynamic Log Ratio Run 3----**

In [42]:
import time
from tqdm import tqdm
import numpy as np
from math import log,e

# === Fixed Parameters ===
leaf_size_rebuild = 2150
epsilon_rebuild = 0.047

epsilon_insert = 0.04

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
kdtree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        kdtree = KDTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        kdtree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        kdtree.epsilon = epsilon_insert
        # Otherwise Insert the batch points using opt node size and epsilon
        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            kdtree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding KD Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    kdtree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = kdtree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of KD Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding KD Tree from scratch with 100,000 points.
Build Time: 0.5359 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 500.80it/s]


Total Search Time: 0.077681 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 122373.18it/s]

Insert Time: 0.0849 sec


...

Total Search Time: 18.803457 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 91398.65it/s]


Insert Time: 1.0978 sec


Querying batch 15: 100%|██████████| 100/100 [00:03<00:00, 31.98it/s]

Total Search Time: 3.709580 sec

=== Summary of KD Tree Scaling Test ===
Points: 100,000, Build Time: 0.5359 sec, Total Search Time: 0.077681 sec, Total Time: 0.6136 sec
Points: 110,000, Insert Time: 0.0849 sec, Total Search Time: 1.242953 sec, Total Time: 1.3279 sec
Points: 120,000, Insert Time: 0.0874 sec, Total Search Time: 0.105642 sec, Total Time: 0.1930 sec
Points: 220,000, Insert Time: 0.9332 sec, Total Search Time: 1.856743 sec, Total Time: 2.7899 sec
Points: 230,000, Insert Time: 0.0906 sec, Total Search Time: 0.197851 sec, Total Time: 0.2885 sec
Points: 240,000, Insert Time: 0.0930 sec, Total Search Time: 0.515191 sec, Total Time: 0.6082 sec
Points: 250,000, Insert Time: 0.0990 sec, Total Search Time: 0.548055 sec, Total Time: 0.6471 sec
Points: 260,000, Insert Time: 0.1490 sec, Total Search Time: 0.572096 sec, Total Time: 0.7211 sec
Points: 360,000, Insert Time: 0.9183 sec, Total Search Time: 0.331513 sec, Total Time: 1.2498 sec
Points: 370,000, Insert Time: 0.0842 sec, Tota

**--------KD dyn log ratio analysis------**

In [43]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 48.0830',
    'Total Time:: 51.3647',
    'Total Time:: 52.9547'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 50.8008 sec, Std Dev: 2.4843 sec, CV: 4.89%


-----------------------------------

**----------VP Hybrid------**

In [46]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**--------vp hybrid sqrt run 1----------**

In [50]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3915 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1283.53it/s]


Total Search Time: 0.026598 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 71016.35it/s]


Insert Time: 0.1436 sec


...

Total Search Time: 0.834516 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 53202.13it/s]


Insert Time: 1.8822 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 375.59it/s]

Total Search Time: 0.400277 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3915 sec, Total Search Time: 0.026598 sec, Total Time: 0.4181 sec
Points: 110,000, Insert Time: 0.1436 sec, Total Search Time: 0.321647 sec, Total Time: 0.4652 sec
Points: 120,000, Insert Time: 0.1668 sec, Total Search Time: 0.047453 sec, Total Time: 0.2143 sec
Points: 220,000, Build Time: 1.1758 sec, Total Search Time: 0.405990 sec, Total Time: 1.5818 sec
Points: 230,000, Insert Time: 0.1629 sec, Total Search Time: 0.026713 sec, Total Time: 0.1896 sec
Points: 240,000, Insert Time: 0.1823 sec, Total Search Time: 0.083005 sec, Total Time: 0.2653 sec
Points: 250,000, Insert Time: 0.1943 sec, Total Search Time: 0.095845 sec, Total Time: 0.2901 sec
Points: 260,000, Insert Time: 0.1888 sec, Total Search Time: 0.120644 sec, Total Time: 0.3095 sec
Points: 360,000, Insert Time: 1.8240 sec, Total Search Time: 0.134667 sec, Total Time: 1.9587 sec
Points: 370,000, Insert Time: 0.1748 sec, Total

**--------vp hybrid sqrt run 2---------**

In [55]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4768 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 606.66it/s]


Total Search Time: 0.052226 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 68729.90it/s]

Insert Time: 0.1494 sec


...

Total Search Time: 0.829017 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 51193.60it/s]


Insert Time: 1.9568 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 335.72it/s]

Total Search Time: 0.446491 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4768 sec, Total Search Time: 0.052226 sec, Total Time: 0.5290 sec
Points: 110,000, Insert Time: 0.1494 sec, Total Search Time: 0.436941 sec, Total Time: 0.5863 sec
Points: 120,000, Insert Time: 0.1543 sec, Total Search Time: 0.050476 sec, Total Time: 0.2047 sec
Points: 220,000, Build Time: 0.9372 sec, Total Search Time: 0.634113 sec, Total Time: 1.5713 sec
Points: 230,000, Insert Time: 0.1457 sec, Total Search Time: 0.030759 sec, Total Time: 0.1765 sec
Points: 240,000, Insert Time: 0.1783 sec, Total Search Time: 0.093784 sec, Total Time: 0.2720 sec
Points: 250,000, Insert Time: 0.1731 sec, Total Search Time: 0.118959 sec, Total Time: 0.2921 sec
Points: 260,000, Insert Time: 0.1715 sec, Total Search Time: 0.134666 sec, Total Time: 0.3062 sec
Points: 360,000, Insert Time: 1.7858 sec, Total Search Time: 0.141998 sec, Total Time: 1.9278 sec
Points: 370,000, Insert Time: 0.1718 sec, Total

**----------vp hybrid sqrt run 3--------**

In [60]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.6278 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1295.50it/s]


Total Search Time: 0.033732 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 66744.70it/s]

Insert Time: 0.1542 sec


...

Total Search Time: 0.702806 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 51627.04it/s]


Insert Time: 1.9416 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 504.17it/s]

Total Search Time: 0.351419 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.6278 sec, Total Search Time: 0.033732 sec, Total Time: 0.6615 sec
Points: 110,000, Insert Time: 0.1542 sec, Total Search Time: 0.506935 sec, Total Time: 0.6611 sec
Points: 120,000, Insert Time: 0.1538 sec, Total Search Time: 0.053637 sec, Total Time: 0.2075 sec
Points: 220,000, Build Time: 1.0868 sec, Total Search Time: 0.410370 sec, Total Time: 1.4971 sec
Points: 230,000, Insert Time: 0.1599 sec, Total Search Time: 0.029283 sec, Total Time: 0.1892 sec
Points: 240,000, Insert Time: 0.1732 sec, Total Search Time: 0.084040 sec, Total Time: 0.2573 sec
Points: 250,000, Insert Time: 0.1724 sec, Total Search Time: 0.104328 sec, Total Time: 0.2767 sec
Points: 260,000, Insert Time: 0.2354 sec, Total Search Time: 0.140222 sec, Total Time: 0.3757 sec
Points: 360,000, Insert Time: 1.6982 sec, Total Search Time: 0.129888 sec, Total Time: 1.8280 sec
Points: 370,000, Insert Time: 0.1680 sec, Total

**--------vp hybrid sqrt analysis---------**

In [65]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 19.1703',
    'Total Time:: 19.4822',
    'Total Time:: 19.5415'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 19.3980 sec, Std Dev: 0.1994 sec, CV: 1.03%


---------------------------

**-----VP hybrid Log Threshold Run 1------**

In [51]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4638 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 890.74it/s]


Total Search Time: 0.036707 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 64986.36it/s]

Insert Time: 0.1582 sec


...

Total Search Time: 0.917812 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 53025.98it/s]


Insert Time: 1.8892 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 370.56it/s]

Total Search Time: 0.414911 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4638 sec, Total Search Time: 0.036707 sec, Total Time: 0.5005 sec
Points: 110,000, Insert Time: 0.1582 sec, Total Search Time: 0.337077 sec, Total Time: 0.4953 sec
Points: 120,000, Insert Time: 0.1940 sec, Total Search Time: 0.045367 sec, Total Time: 0.2394 sec
Points: 220,000, Insert Time: 1.5571 sec, Total Search Time: 1.047824 sec, Total Time: 2.6049 sec
Points: 230,000, Insert Time: 0.1549 sec, Total Search Time: 0.120263 sec, Total Time: 0.2752 sec
Points: 240,000, Insert Time: 0.1547 sec, Total Search Time: 0.308482 sec, Total Time: 0.4632 sec
Points: 250,000, Insert Time: 0.1521 sec, Total Search Time: 0.360965 sec, Total Time: 0.5130 sec
Points: 260,000, Insert Time: 0.1788 sec, Total Search Time: 0.426295 sec, Total Time: 0.6051 sec
Points: 360,000, Insert Time: 1.5813 sec, Total Search Time: 0.260587 sec, Total Time: 1.8419 sec
Points: 370,000, Insert Time: 0.1624 sec, Tota

**-------VP Hybrid log Run 2-----**

In [56]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4126 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 720.41it/s]


Total Search Time: 0.043356 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 56300.14it/s]


Insert Time: 0.1820 sec


...

Total Search Time: 0.832597 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 52030.51it/s]


Insert Time: 1.9245 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 409.85it/s]

Total Search Time: 0.393590 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4126 sec, Total Search Time: 0.043356 sec, Total Time: 0.4559 sec
Points: 110,000, Insert Time: 0.1820 sec, Total Search Time: 0.482096 sec, Total Time: 0.6641 sec
Points: 120,000, Insert Time: 0.1610 sec, Total Search Time: 0.049380 sec, Total Time: 0.2104 sec
Points: 220,000, Insert Time: 1.5554 sec, Total Search Time: 1.148896 sec, Total Time: 2.7043 sec
Points: 230,000, Insert Time: 0.1517 sec, Total Search Time: 0.152429 sec, Total Time: 0.3042 sec
Points: 240,000, Insert Time: 0.1595 sec, Total Search Time: 0.322126 sec, Total Time: 0.4817 sec
Points: 250,000, Insert Time: 0.1703 sec, Total Search Time: 0.382477 sec, Total Time: 0.5528 sec
Points: 260,000, Insert Time: 0.1647 sec, Total Search Time: 0.408779 sec, Total Time: 0.5734 sec
Points: 360,000, Insert Time: 1.5029 sec, Total Search Time: 0.242628 sec, Total Time: 1.7456 sec
Points: 370,000, Insert Time: 0.1555 sec, Tota

**-----VP Hybrid log Run 3--------**

In [61]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.6201 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1095.81it/s]


Total Search Time: 0.038186 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 67384.66it/s]

Insert Time: 0.1518 sec


...

Total Search Time: 0.774693 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 52613.80it/s]


Insert Time: 1.9043 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 531.53it/s]

Total Search Time: 0.333181 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.6201 sec, Total Search Time: 0.038186 sec, Total Time: 0.6582 sec
Points: 110,000, Insert Time: 0.1518 sec, Total Search Time: 0.505805 sec, Total Time: 0.6576 sec
Points: 120,000, Insert Time: 0.1566 sec, Total Search Time: 0.055351 sec, Total Time: 0.2120 sec
Points: 220,000, Insert Time: 1.4680 sec, Total Search Time: 1.347472 sec, Total Time: 2.8154 sec
Points: 230,000, Insert Time: 0.1906 sec, Total Search Time: 0.189685 sec, Total Time: 0.3803 sec
Points: 240,000, Insert Time: 0.1606 sec, Total Search Time: 0.372438 sec, Total Time: 0.5330 sec
Points: 250,000, Insert Time: 0.1687 sec, Total Search Time: 0.381259 sec, Total Time: 0.5499 sec
Points: 260,000, Insert Time: 0.1491 sec, Total Search Time: 0.382316 sec, Total Time: 0.5314 sec
Points: 360,000, Insert Time: 1.4974 sec, Total Search Time: 0.238093 sec, Total Time: 1.7355 sec
Points: 370,000, Insert Time: 0.1491 sec, Tota

**-------vp hybrid log analysis---------**

In [64]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 21.7936',
    'Total Time:: 22.0719',
    'Total Time:: 21.8966'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 21.9207 sec, Std Dev: 0.1407 sec, CV: 0.64%


------------------------

**-----VP hybrid Log Ratio Run 1-------**

In [52]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3867 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1215.79it/s]


Total Search Time: 0.030207 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 65850.95it/s]


Insert Time: 0.1564 sec


...

Total Search Time: 0.833279 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 54735.90it/s]


Insert Time: 1.8300 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 341.47it/s]

Total Search Time: 0.428357 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3867 sec, Total Search Time: 0.030207 sec, Total Time: 0.4169 sec
Points: 110,000, Insert Time: 0.1564 sec, Total Search Time: 0.315379 sec, Total Time: 0.4718 sec
Points: 120,000, Insert Time: 0.1552 sec, Total Search Time: 0.049268 sec, Total Time: 0.2044 sec
Points: 220,000, Insert Time: 1.5723 sec, Total Search Time: 1.137762 sec, Total Time: 2.7100 sec
Points: 230,000, Insert Time: 0.1719 sec, Total Search Time: 0.128691 sec, Total Time: 0.3006 sec
Points: 240,000, Insert Time: 0.1534 sec, Total Search Time: 0.307351 sec, Total Time: 0.4607 sec
Points: 250,000, Insert Time: 0.1749 sec, Total Search Time: 0.337909 sec, Total Time: 0.5128 sec
Points: 260,000, Insert Time: 0.1582 sec, Total Search Time: 0.374358 sec, Total Time: 0.5326 sec
Points: 360,000, Insert Time: 1.4889 sec, Total Search Time: 0.248143 sec, Total Time: 1.7370 sec
Points: 370,000, Insert Time: 0.1527 sec, Tota

**------VP Hybrid Log Ratio Run 2---------**

In [57]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3812 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 649.90it/s]


Total Search Time: 0.048526 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 63207.21it/s]


Insert Time: 0.1623 sec


...

Total Search Time: 0.838869 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 53148.89it/s]


Insert Time: 1.8852 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 396.83it/s]

Total Search Time: 0.402879 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3812 sec, Total Search Time: 0.048526 sec, Total Time: 0.4298 sec
Points: 110,000, Insert Time: 0.1623 sec, Total Search Time: 0.433453 sec, Total Time: 0.5957 sec
Points: 120,000, Insert Time: 0.1594 sec, Total Search Time: 0.054219 sec, Total Time: 0.2136 sec
Points: 220,000, Insert Time: 1.5964 sec, Total Search Time: 1.289099 sec, Total Time: 2.8855 sec
Points: 230,000, Insert Time: 0.1618 sec, Total Search Time: 0.144173 sec, Total Time: 0.3060 sec
Points: 240,000, Insert Time: 0.1631 sec, Total Search Time: 0.309904 sec, Total Time: 0.4731 sec
Points: 250,000, Insert Time: 0.1439 sec, Total Search Time: 0.362401 sec, Total Time: 0.5063 sec
Points: 260,000, Insert Time: 0.1517 sec, Total Search Time: 0.370960 sec, Total Time: 0.5226 sec
Points: 360,000, Insert Time: 1.4892 sec, Total Search Time: 0.242279 sec, Total Time: 1.7315 sec
Points: 370,000, Insert Time: 0.1495 sec, Tota

**------VP Hybrid Log Ratio Run 3----------**

In [62]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 9300
epsilon_rebuild = 10.0

leaf_size_insert = 800
epsilon_insert = 8.0

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.leaf_size = leaf_size_insert
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.6089 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1165.75it/s]


Total Search Time: 0.037284 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 62441.91it/s]


Insert Time: 0.1686 sec


...

Total Search Time: 0.660806 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 51606.51it/s]


Insert Time: 1.9428 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 508.53it/s]

Total Search Time: 0.365522 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.6089 sec, Total Search Time: 0.037284 sec, Total Time: 0.6461 sec
Points: 110,000, Insert Time: 0.1686 sec, Total Search Time: 0.530102 sec, Total Time: 0.6987 sec
Points: 120,000, Insert Time: 0.1864 sec, Total Search Time: 0.063149 sec, Total Time: 0.2496 sec
Points: 220,000, Insert Time: 1.6120 sec, Total Search Time: 1.240778 sec, Total Time: 2.8528 sec
Points: 230,000, Insert Time: 0.1453 sec, Total Search Time: 0.147396 sec, Total Time: 0.2927 sec
Points: 240,000, Insert Time: 0.1511 sec, Total Search Time: 0.355223 sec, Total Time: 0.5064 sec
Points: 250,000, Insert Time: 0.1631 sec, Total Search Time: 0.392479 sec, Total Time: 0.5556 sec
Points: 260,000, Insert Time: 0.1506 sec, Total Search Time: 0.385251 sec, Total Time: 0.5359 sec
Points: 360,000, Insert Time: 1.4927 sec, Total Search Time: 0.231804 sec, Total Time: 1.7245 sec
Points: 370,000, Insert Time: 0.1463 sec, Tota

**-----vp hybrid log ratio analysis----------**

In [63]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 21.8303',
    'Total Time:: 21.4663',
    'Total Time:: 21.8842'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 21.7269 sec, Std Dev: 0.2273 sec, CV: 1.05%


-----------------------------------

**-------VP Tree Epsilon Dynamic-------**

In [66]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

**--------vp dyn sqrt run 1---------**

In [70]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3385 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3518.86it/s]


Total Search Time: 0.012450 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 76292.12it/s]


Insert Time: 0.1347 sec


...

Total Search Time: 0.778990 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69881.40it/s]


Insert Time: 1.4358 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 373.13it/s]

Total Search Time: 0.397632 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3385 sec, Total Search Time: 0.012450 sec, Total Time: 0.3509 sec
Points: 110,000, Insert Time: 0.1347 sec, Total Search Time: 0.160529 sec, Total Time: 0.2953 sec
Points: 120,000, Insert Time: 0.1365 sec, Total Search Time: 0.026623 sec, Total Time: 0.1631 sec
Points: 220,000, Build Time: 0.7914 sec, Total Search Time: 0.077134 sec, Total Time: 0.8686 sec
Points: 230,000, Insert Time: 0.1376 sec, Total Search Time: 0.015372 sec, Total Time: 0.1530 sec
Points: 240,000, Insert Time: 0.1399 sec, Total Search Time: 0.055563 sec, Total Time: 0.1954 sec
Points: 250,000, Insert Time: 0.1511 sec, Total Search Time: 0.072731 sec, Total Time: 0.2238 sec
Points: 260,000, Insert Time: 0.1371 sec, Total Search Time: 0.088455 sec, Total Time: 0.2255 sec
Points: 360,000, Insert Time: 1.4283 sec, Total Search Time: 0.142765 sec, Total Time: 1.5711 sec
Points: 370,000, Insert Time: 0.1412 sec, Total

**----------vp dyn sqrt run 2-------**

In [75]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3177 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1438.18it/s]


Total Search Time: 0.019608 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 82283.20it/s]


Insert Time: 0.1246 sec


...

Total Search Time: 0.834871 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70250.02it/s]


Insert Time: 1.4270 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 324.77it/s]

Total Search Time: 0.443927 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3177 sec, Total Search Time: 0.019608 sec, Total Time: 0.3373 sec
Points: 110,000, Insert Time: 0.1246 sec, Total Search Time: 0.221650 sec, Total Time: 0.3463 sec
Points: 120,000, Insert Time: 0.1239 sec, Total Search Time: 0.028732 sec, Total Time: 0.1526 sec
Points: 220,000, Build Time: 0.7501 sec, Total Search Time: 0.070034 sec, Total Time: 0.8201 sec
Points: 230,000, Insert Time: 0.1335 sec, Total Search Time: 0.016545 sec, Total Time: 0.1501 sec
Points: 240,000, Insert Time: 0.1323 sec, Total Search Time: 0.064456 sec, Total Time: 0.1967 sec
Points: 250,000, Insert Time: 0.1332 sec, Total Search Time: 0.085891 sec, Total Time: 0.2191 sec
Points: 260,000, Insert Time: 0.1320 sec, Total Search Time: 0.105406 sec, Total Time: 0.2374 sec
Points: 360,000, Insert Time: 1.3512 sec, Total Search Time: 0.133912 sec, Total Time: 1.4851 sec
Points: 370,000, Insert Time: 0.1408 sec, Total

**--------vp dyn sqrt run 3---------**

In [80]:
import time
from tqdm import tqdm
import numpy as np
from math import sqrt

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = points_at_last_rebuild + int(sqrt(points_at_last_rebuild)) if points_at_last_rebuild > 0 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3807 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 2555.17it/s]


Total Search Time: 0.013310 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 74521.42it/s]


Insert Time: 0.1372 sec


...

Total Search Time: 0.792430 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69927.36it/s]


Insert Time: 1.4322 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 380.51it/s]

Total Search Time: 0.421110 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3807 sec, Total Search Time: 0.013310 sec, Total Time: 0.3940 sec
Points: 110,000, Insert Time: 0.1372 sec, Total Search Time: 0.122899 sec, Total Time: 0.2601 sec
Points: 120,000, Insert Time: 0.1395 sec, Total Search Time: 0.021758 sec, Total Time: 0.1613 sec
Points: 220,000, Build Time: 0.8305 sec, Total Search Time: 0.291014 sec, Total Time: 1.1215 sec
Points: 230,000, Insert Time: 0.1404 sec, Total Search Time: 0.018918 sec, Total Time: 0.1593 sec
Points: 240,000, Insert Time: 0.1414 sec, Total Search Time: 0.061045 sec, Total Time: 0.2025 sec
Points: 250,000, Insert Time: 0.1417 sec, Total Search Time: 0.078424 sec, Total Time: 0.2202 sec
Points: 260,000, Insert Time: 0.1458 sec, Total Search Time: 0.097879 sec, Total Time: 0.2437 sec
Points: 360,000, Insert Time: 1.3734 sec, Total Search Time: 0.129714 sec, Total Time: 1.5031 sec
Points: 370,000, Insert Time: 0.1390 sec, Total

**-------vp dyn sqrt analysis---------**

In [85]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 14.8171',
    'Total Time:: 14.2421',
    'Total Time:: 15.2696'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 14.7763 sec, Std Dev: 0.5150 sec, CV: 3.49%


------------------------------------

**-----VP  Dynamic Log  Run 1------**

In [71]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3703 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3808.85it/s]


Total Search Time: 0.010641 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 75691.83it/s]


Insert Time: 0.1347 sec


...

Total Search Time: 0.760701 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70483.62it/s]


Insert Time: 1.4215 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 390.29it/s]

Total Search Time: 0.381708 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3703 sec, Total Search Time: 0.010641 sec, Total Time: 0.3810 sec
Points: 110,000, Insert Time: 0.1347 sec, Total Search Time: 0.163261 sec, Total Time: 0.2980 sec
Points: 120,000, Insert Time: 0.1331 sec, Total Search Time: 0.027015 sec, Total Time: 0.1601 sec
Points: 220,000, Insert Time: 1.3633 sec, Total Search Time: 0.961556 sec, Total Time: 2.3248 sec
Points: 230,000, Insert Time: 0.1330 sec, Total Search Time: 0.122887 sec, Total Time: 0.2559 sec
Points: 240,000, Insert Time: 0.1357 sec, Total Search Time: 0.294200 sec, Total Time: 0.4299 sec
Points: 250,000, Insert Time: 0.1370 sec, Total Search Time: 0.338758 sec, Total Time: 0.4758 sec
Points: 260,000, Insert Time: 0.1367 sec, Total Search Time: 0.341210 sec, Total Time: 0.4779 sec
Points: 360,000, Insert Time: 1.3472 sec, Total Search Time: 0.244143 sec, Total Time: 1.5913 sec
Points: 370,000, Insert Time: 0.1345 sec, Tota

**--------VP Dynamic log Run 2--------**

In [76]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4850 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1540.27it/s]


Total Search Time: 0.018693 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 83526.25it/s]


Insert Time: 0.1234 sec


...

Total Search Time: 0.841027 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69775.10it/s]


Insert Time: 1.4357 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 347.01it/s]

Total Search Time: 0.425128 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4850 sec, Total Search Time: 0.018693 sec, Total Time: 0.5037 sec
Points: 110,000, Insert Time: 0.1234 sec, Total Search Time: 0.237461 sec, Total Time: 0.3609 sec
Points: 120,000, Insert Time: 0.1262 sec, Total Search Time: 0.033698 sec, Total Time: 0.1599 sec
Points: 220,000, Insert Time: 1.2342 sec, Total Search Time: 1.212284 sec, Total Time: 2.4465 sec
Points: 230,000, Insert Time: 0.1248 sec, Total Search Time: 0.134812 sec, Total Time: 0.2596 sec
Points: 240,000, Insert Time: 0.1295 sec, Total Search Time: 0.367242 sec, Total Time: 0.4967 sec
Points: 250,000, Insert Time: 0.1266 sec, Total Search Time: 0.387236 sec, Total Time: 0.5138 sec
Points: 260,000, Insert Time: 0.1221 sec, Total Search Time: 0.417844 sec, Total Time: 0.5400 sec
Points: 360,000, Insert Time: 1.2171 sec, Total Search Time: 0.266667 sec, Total Time: 1.4838 sec
Points: 370,000, Insert Time: 0.1271 sec, Tota


**------VP Dynamic log Run 3----------**

In [81]:
import time
from tqdm import tqdm
import numpy as np
from math import log

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    threshold = int(points_at_last_rebuild * log(points_at_last_rebuild)) if points_at_last_rebuild > 1 else 0
    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3837 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3243.98it/s]


Total Search Time: 0.011005 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 72759.80it/s]


Insert Time: 0.1398 sec


...

Total Search Time: 0.828455 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 71162.18it/s]


Insert Time: 1.4093 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 419.60it/s]

Total Search Time: 0.389051 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3837 sec, Total Search Time: 0.011005 sec, Total Time: 0.3947 sec
Points: 110,000, Insert Time: 0.1398 sec, Total Search Time: 0.128299 sec, Total Time: 0.2681 sec
Points: 120,000, Insert Time: 0.1455 sec, Total Search Time: 0.026240 sec, Total Time: 0.1717 sec
Points: 220,000, Insert Time: 1.3601 sec, Total Search Time: 0.792688 sec, Total Time: 2.1528 sec
Points: 230,000, Insert Time: 0.1366 sec, Total Search Time: 0.124505 sec, Total Time: 0.2611 sec
Points: 240,000, Insert Time: 0.1400 sec, Total Search Time: 0.261382 sec, Total Time: 0.4014 sec
Points: 250,000, Insert Time: 0.1387 sec, Total Search Time: 0.278055 sec, Total Time: 0.4167 sec
Points: 260,000, Insert Time: 0.1410 sec, Total Search Time: 0.316188 sec, Total Time: 0.4572 sec
Points: 360,000, Insert Time: 1.3649 sec, Total Search Time: 0.228808 sec, Total Time: 1.5937 sec
Points: 370,000, Insert Time: 0.1359 sec, Tota

**--------vp dyn log analysis----------**

In [84]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 17.7592',
    'Total Time:: 17.7265',
    'Total Time:: 17.7124'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 17.7327 sec, Std Dev: 0.0240 sec, CV: 0.14%


--------------------------------

**-----VP  Dynamic Log Ratio Run 1-----**

In [72]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3212 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3413.61it/s]


Total Search Time: 0.012465 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 75211.35it/s]


Insert Time: 0.1356 sec


...

Total Search Time: 0.765536 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69807.59it/s]


Insert Time: 1.4361 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 380.97it/s]

Total Search Time: 0.406811 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3212 sec, Total Search Time: 0.012465 sec, Total Time: 0.3337 sec
Points: 110,000, Insert Time: 0.1356 sec, Total Search Time: 0.170385 sec, Total Time: 0.3060 sec
Points: 120,000, Insert Time: 0.1420 sec, Total Search Time: 0.027420 sec, Total Time: 0.1694 sec
Points: 220,000, Insert Time: 1.3942 sec, Total Search Time: 0.966672 sec, Total Time: 2.3609 sec
Points: 230,000, Insert Time: 0.1377 sec, Total Search Time: 0.125052 sec, Total Time: 0.2627 sec
Points: 240,000, Insert Time: 0.1355 sec, Total Search Time: 0.290291 sec, Total Time: 0.4258 sec
Points: 250,000, Insert Time: 0.1351 sec, Total Search Time: 0.335938 sec, Total Time: 0.4710 sec
Points: 260,000, Insert Time: 0.1366 sec, Total Search Time: 0.345229 sec, Total Time: 0.4818 sec
Points: 360,000, Insert Time: 1.3643 sec, Total Search Time: 0.245902 sec, Total Time: 1.6102 sec
Points: 370,000, Insert Time: 0.1363 sec, Tota

**--------VP Dyn Log Ratio Run 2-------**

In [77]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.4861 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 1481.62it/s]


Total Search Time: 0.021073 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 81469.85it/s]


Insert Time: 0.1256 sec


...

Total Search Time: 0.882136 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 70718.90it/s]


Insert Time: 1.4173 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 338.73it/s]

Total Search Time: 0.435539 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.4861 sec, Total Search Time: 0.021073 sec, Total Time: 0.5072 sec
Points: 110,000, Insert Time: 0.1256 sec, Total Search Time: 0.225333 sec, Total Time: 0.3510 sec
Points: 120,000, Insert Time: 0.1296 sec, Total Search Time: 0.031107 sec, Total Time: 0.1607 sec
Points: 220,000, Insert Time: 1.2195 sec, Total Search Time: 1.210794 sec, Total Time: 2.4303 sec
Points: 230,000, Insert Time: 0.1270 sec, Total Search Time: 0.136865 sec, Total Time: 0.2639 sec
Points: 240,000, Insert Time: 0.1256 sec, Total Search Time: 0.363355 sec, Total Time: 0.4890 sec
Points: 250,000, Insert Time: 0.1254 sec, Total Search Time: 0.386605 sec, Total Time: 0.5120 sec
Points: 260,000, Insert Time: 0.1278 sec, Total Search Time: 0.409460 sec, Total Time: 0.5372 sec
Points: 360,000, Insert Time: 1.2161 sec, Total Search Time: 0.259974 sec, Total Time: 1.4761 sec
Points: 370,000, Insert Time: 0.1403 sec, Tota

**-------VP Dyn Log Ratio Run 3--------**

In [82]:
import time
from tqdm import tqdm
import numpy as np
from math import log, e

# === Fixed Parameters ===
leaf_size_rebuild = 8700
epsilon_rebuild = 17.0

epsilon_insert = 9.5

# Bookkeeping
points_at_last_rebuild = 0
accumulated_since_rebuild = 0
batch_idx = 0
results = []

# The total time needed to complete all tasks
grand_total_time = 0.0

cumulative_data = []
vptree = None  # Will be created on first build

for data_size, _ in test_combinations:
    print(f"\nBatch {batch_idx+1}: Adding {data_size:,} points")

    # Slice batch
    start_idx = sum(size for size, _ in test_combinations[:batch_idx])
    end_idx = start_idx + data_size
    new_batch = combined_dataset[start_idx:end_idx]
    batch_size = len(new_batch)
    accumulated_since_rebuild += batch_size

    # Determine rebuild threshold
    if points_at_last_rebuild > e:
        threshold = int(points_at_last_rebuild * log(points_at_last_rebuild) / log(log(points_at_last_rebuild)))
    else:
        threshold = 0

    rebuilt = False
    insert_time = 0.0
    build_time = 0.0

    if accumulated_since_rebuild >= threshold:
        rebuilt = True
        # Rebuild with old data + new batch
        if len(cumulative_data):
            cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)
        else:
            cumulative_data = new_batch

        build_start = time.perf_counter()
        vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
        vptree.build(cumulative_data)
        build_time = time.perf_counter() - build_start

        points_at_last_rebuild = len(cumulative_data)
        accumulated_since_rebuild = 0
    else:
        # Insert using the appropriately optimized leaf and epsilon sizes
        vptree.epsilon = epsilon_insert

        insert_start = time.perf_counter()
        for point in tqdm(new_batch, desc=f"Inserting {batch_size:,} points"):
            vptree.insert(point)
        insert_time = time.perf_counter() - insert_start

        cumulative_data = np.concatenate((cumulative_data, new_batch), axis=0)

    # Output build/insert time
    if rebuilt:
        print(f"Rebuilding VP Tree from scratch with {points_at_last_rebuild:,} points.")
        print(f"Build Time: {build_time:.4f} sec")
    else:
        print(f"Insert Time: {insert_time:.4f} sec")

    # Retrieve the pre-generated queries from the list using the same order.
    queries = queries_list[batch_idx]


    search_start = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying batch {batch_idx+1}"):
        r_min, nn_pt = vptree.search(query=query)
    search_end = time.perf_counter()
    total_search_time = search_end - search_start

    print(f"Total Search Time: {total_search_time:.6f} sec")

    # Total build/insert + search time
    total_build_or_insert_time = build_time + insert_time
    total_batch_time = total_build_or_insert_time + total_search_time
    grand_total_time += total_batch_time

    # Store results
    results.append((
        len(cumulative_data),  # total points
        build_time,
        insert_time,
        total_search_time,
        total_batch_time
    ))

    batch_idx += 1

# === Summary ===
print("\n=== Summary of VP Tree Scaling Test ===")
for points, b_time, i_time, s_time, total_time in results:
    if b_time > 0:
        print(f"Points: {points:,}, Build Time: {b_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")
    else:
        print(f"Points: {points:,}, Insert Time: {i_time:.4f} sec, Total Search Time: {s_time:.6f} sec, Total Time: {total_time:.4f} sec")

print(f"\nTotal Time:: {grand_total_time:.4f} sec")


Batch 1: Adding 100,000 points
Rebuilding VP Tree from scratch with 100,000 points.
Build Time: 0.3829 sec


Querying batch 1: 100%|██████████| 20/20 [00:00<00:00, 3061.65it/s]


Total Search Time: 0.011294 sec

Batch 2: Adding 10,000 points


Inserting 10,000 points: 100%|██████████| 10000/10000 [00:00<00:00, 75369.61it/s]


Insert Time: 0.1355 sec


...

Total Search Time: 0.803004 sec

Batch 15: Adding 100,000 points


Inserting 100,000 points: 100%|██████████| 100000/100000 [00:01<00:00, 69869.58it/s]


Insert Time: 1.4348 sec


Querying batch 15: 100%|██████████| 100/100 [00:00<00:00, 421.41it/s]

Total Search Time: 0.371191 sec

=== Summary of VP Tree Scaling Test ===
Points: 100,000, Build Time: 0.3829 sec, Total Search Time: 0.011294 sec, Total Time: 0.3942 sec
Points: 110,000, Insert Time: 0.1355 sec, Total Search Time: 0.117156 sec, Total Time: 0.2527 sec
Points: 120,000, Insert Time: 0.1377 sec, Total Search Time: 0.025545 sec, Total Time: 0.1633 sec
Points: 220,000, Insert Time: 1.3730 sec, Total Search Time: 0.830020 sec, Total Time: 2.2030 sec
Points: 230,000, Insert Time: 0.1492 sec, Total Search Time: 0.122595 sec, Total Time: 0.2718 sec
Points: 240,000, Insert Time: 0.1383 sec, Total Search Time: 0.257586 sec, Total Time: 0.3959 sec
Points: 250,000, Insert Time: 0.1511 sec, Total Search Time: 0.281005 sec, Total Time: 0.4321 sec
Points: 260,000, Insert Time: 0.1360 sec, Total Search Time: 0.316880 sec, Total Time: 0.4528 sec
Points: 360,000, Insert Time: 1.4002 sec, Total Search Time: 0.223053 sec, Total Time: 1.6232 sec
Points: 370,000, Insert Time: 0.1356 sec, Tota

**-------vp dyn log ratio analysis--------**

In [83]:
import re
import numpy as np

# Your 3 result strings
results = [
    'Total Time:: 17.5218',
    'Total Time:: 17.6561',
    'Total Time:: 17.7091'
]

# Extract times using regex
total_accumulated_time = []

for line in results:
    match = re.search(r'Total Time::\s*([\d.]+)', line)
    if match:
        total_accumulated_time.append(float(match.group(1)))
    else:
        print(f"Warning: No match found in line: {line}")

# Compute mean, std, and CV
def stats(name, values):
    avg = np.mean(values)
    std = np.std(values, ddof=1)  # Sample std dev
    cv = (std / avg) * 100 if avg != 0 else float('nan')  # CV in percentage
    print(f"{name} - Mean: {avg:.4f} sec, Std Dev: {std:.4f} sec, CV: {cv:.2f}%")

stats("Total Time", total_accumulated_time)


Total Time - Mean: 17.6290 sec, Std Dev: 0.0965 sec, CV: 0.55%
